In [1]:
#Imports
import numpy as np
import pandas as pd
import geopandas as gpd
import difflib
import matplotlib.pyplot as plt
import time

def  processEvents(dataFilepath_sim, nrows):
    PTs = []
    PEVs = []
    print('read', dataFilepath_sim)
    for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):
        if sum((chunk['type'] == 'PathTraversal')) > 0:
            chunk['vehicle'] = chunk['vehicle'].astype(str)
            PT = chunk.loc[(chunk['type'] == 'PathTraversal') & (chunk['length'] > 0)].dropna(how='all', axis=1)
            PT['departureTime'] = PT['departureTime'].astype(int)
            PT['arrivalTime'] = PT['arrivalTime'].astype(int)

            PTs.append(PT[['driver', 'vehicle', 'mode', 'length', 'startX', 'startY', 'endX', 'endY', 'vehicleType',
                           'arrivalTime', 'departureTime', 'primaryFuel', 'primaryFuelType', 'secondaryFuel',
                           'secondaryFuelType', 'numPassengers', 'riders','time']])
            print(chunk.type.value_counts())
            PEV = chunk.loc[(chunk.type == "PersonEntersVehicle") &
                            ~(chunk['person'].apply(str).str.contains('Agent').fillna(False)) &
                            ~(chunk['vehicle'].str.contains('body').fillna(False)), :].dropna(how='all', axis=1)
            if ~PEV.empty:
                PEV['time'] = PEV['time'].astype(int)
                PEVs.append(PEV)

    Events_file_sim_PT = pd.concat(PTs)[['vehicle','time','endY','endX','startY','startX','mode']]
    Events_file_sim_PE = pd.concat(PEVs)[['vehicle','time',]]
    print(Events_file_sim_PE)

    return Events_file_sim_PT, Events_file_sim_PE



#Filter transit trips
def filter_transit(Events_file_sim_PT,Events_file_sim_PE):
    print(Events_file_sim_PT['mode'].value_counts())
    Events_file_sim_PT = Events_file_sim_PT[(Events_file_sim_PT['mode'] == 'bus')|
                                            (Events_file_sim_PT['mode'] == 'tram')|
                                            (Events_file_sim_PT['mode'] == 'subway')|
                                            (Events_file_sim_PT['mode'] == 'cable_car')|
                                            (Events_file_sim_PT['mode'] == 'ferry')|
                                            (Events_file_sim_PT['mode'] == 'rail')]
    print(Events_file_sim_PT['mode'].value_counts())
    Events_file_sim_PE = Events_file_sim_PE[Events_file_sim_PE['vehicle'].isin(Events_file_sim_PT['vehicle'])]
    
    return Events_file_sim_PE

def guess_agency(Events_file_sim_PE):

    agencies = []
    for vehicleID in Events_file_sim_PE['vehicle']:
        agency = vehicleID.split(':')[0]
        if agency == 'petalumatransit-petaluma-ca-us':
            agencies.append('PE')
        elif agency == 'westcat-ca-us':
            agencies.append('WC')
        elif agency == 'caltrain-ca-us':
            agencies.append('CA')
        elif agency == 'riovista-ca-us':
            agencies.append('RV')
        elif agency == 'unioncity-ca-us':
            agencies.append('UC')
        else:
            if len(agency) == 2:
                agencies.append(agency)
            elif agency == 'Caltrain':
                agencies.append('CA')
            else:
                print('Warning, this agency is not recognized:', agency)
    Events_file_sim_PE['agency'] = agencies

    print(np.unique(agencies))
    
    return Events_file_sim_PE

def guess_route(Events_file_sim_PE, GTFS_filepaths):

    GTFS_trip_files = {}

    for GTFS_filepath, GTFS in zip(GTFS_filepaths,GTFSs):
        GTFS_trip_files[GTFS] = pd.read_csv(GTFS_filepath+'trips.txt')

    route_ids = []
    total_routes = len(Events_file_sim_PE['vehicle'])
    i = 0
    time_start = time.time()
    for vehicle, agency in zip(Events_file_sim_PE['vehicle'],Events_file_sim_PE['agency']):
        i+=1
        if i%10000 ==0:
            print(i,'/',total_routes,'. Time = ', time.time()-time_start, '. Estimated remaining time:', (time.time()-time_start)/i*total_routes-(time.time()-time_start))
        if agency == 'SM':
            route_ids.append(agency+':'+str(list(GTFS_trip_files[agency]['route_id'][
                GTFS_trip_files[agency]['trip_id'].astype(str)==
                     str(vehicle).split(':')[1]+'|'
                     +str(vehicle).split(':')[2]+':'
                     +str(vehicle).split(':')[3]+'|'
                     +str(vehicle).split(':')[4]+':'
                     +str(vehicle).split(':')[5]+':'
                     +str(vehicle).split(':')[6]])[0]))
        else:
#             print(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id'].astype(str)==str(vehicle.split(':')[1])])[0])
            try:
                route_ids.append(agency+':'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id'].astype(str)==str(vehicle.split(':')[1])])[0]))
            except:
                print('Warning, trip non found for vehicle', vehicle)
                route_ids.append('tripID not found')
    #     elif agency == 'GG':
    #         route_ids.append('GG:'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id']==vehicle.split(':')[1]])[0]))
    #     elif agency == 'SF':
    #         route_ids.append('SF:'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id']==vehicle.split(':')[1]])[0]))
    Events_file_sim_PE['route_id'] = route_ids
    
    return Events_file_sim_PE











#####################################################################################
# dataFilepath_sim = [
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#             ]

                    
# years = ['2018','2019','2020','2021']
#####################################################################################
dataFilepath_sim = [
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
                's3://beam-outputs/pilates-outputs/sfbay-baseline-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
                's3://beam-outputs/pilates-outputs/test_49_3/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
            ]

                    
years = ['2018']

names = []
for year in years:
    names.append('Baseline'+year)
    names.append('Future'+year)

output_filepath = '/Users/cpoliziani/Downloads/Transit Rich/Results/'

GTFS_filepath = '/Users/cpoliziani/Downloads/Data/TR/GTFS/r5-simple-no-local/'
GTFS_filepath2 = '/Users/cpoliziani/Downloads/Data/TR/GTFS/r5-simple-no-local-TR-49/'

RA_output = 'agency_ridershipNew3.csv'
RR_output = 'route_ridershipNew3.csv'

GTFSs = ['3D','AC','AM','AY','BA','CA','CC','CE','CM','CT','DE','EM','GG','HF','MA','PE','RV', 'SB',
        'SC','SF','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH']

GTFS_3D = GTFS_filepath+'3D/'
GTFS_AC = GTFS_filepath+'AC/'
GTFS_AM = GTFS_filepath+'AM/'
GTFS_AY = GTFS_filepath+'AY/'
GTFS_BA = GTFS_filepath+'BA/'
GTFS_Caltrain = GTFS_filepath+'Caltrain/'
GTFS_CC = GTFS_filepath+'CC/'
GTFS_CE = GTFS_filepath+'CE/'
GTFS_CM = GTFS_filepath+'CM/'
GTFS_CT = GTFS_filepath+'CT/'
GTFS_DE = GTFS_filepath+'DE/'
GTFS_EM = GTFS_filepath+'EM/'
GTFS_GG = GTFS_filepath+'GG/'
GTFS_HF = GTFS_filepath+'HF/'
GTFS_MA = GTFS_filepath+'MA/'
GTFS_PE = GTFS_filepath+'PE/'
GTFS_RV = GTFS_filepath+'RV/'
GTFS_SB = GTFS_filepath+'SB/'
GTFS_SC = GTFS_filepath+'SC/'
GTFS_SF = GTFS_filepath+'SF/'
GTFS_SM = GTFS_filepath+'SM/'
GTFS_SO = GTFS_filepath+'SO/'
GTFS_SR = GTFS_filepath+'SR/'
GTFS_ST = GTFS_filepath+'ST/'
GTFS_TD = GTFS_filepath+'TD/'
GTFS_UC = GTFS_filepath+'UC/'
GTFS_VC = GTFS_filepath+'VC/'
GTFS_VN = GTFS_filepath+'VN/'
GTFS_VT = GTFS_filepath+'VT/'
GTFS_WC = GTFS_filepath+'WC/'
GTFS_WH = GTFS_filepath+'WH/'


GTFS_3D2 = GTFS_filepath2+'3D/'
GTFS_AC2 = GTFS_filepath2+'AC/'
GTFS_AM2 = GTFS_filepath2+'AM/'
GTFS_AY2 = GTFS_filepath2+'AY/'
GTFS_BA2 = GTFS_filepath2+'BA/'
GTFS_Caltrain2 = GTFS_filepath2+'Caltrain/'
GTFS_CC2 = GTFS_filepath2+'CC/'
GTFS_CE2 = GTFS_filepath2+'CE/'
GTFS_CM2 = GTFS_filepath2+'CM/'
GTFS_CT2 = GTFS_filepath2+'CT/'
GTFS_DE2 = GTFS_filepath2+'DE/'
GTFS_EM2 = GTFS_filepath2+'EM/'
GTFS_GG2 = GTFS_filepath2+'GG/'
GTFS_HF2 = GTFS_filepath2+'HF/'
GTFS_MA2 = GTFS_filepath2+'MA/'
GTFS_PE2 = GTFS_filepath2+'PE/'
GTFS_RV2 = GTFS_filepath2+'RV/'
GTFS_SB2 = GTFS_filepath2+'SB/'
GTFS_SC2 = GTFS_filepath2+'SC/'
GTFS_SF2 = GTFS_filepath2+'SF/'
GTFS_SM2 = GTFS_filepath2+'SM/'
GTFS_SO2 = GTFS_filepath2+'SO/'
GTFS_SR2 = GTFS_filepath2+'SR/'
GTFS_ST2 = GTFS_filepath2+'ST/'
GTFS_TD2 = GTFS_filepath2+'TD/'
GTFS_UC2 = GTFS_filepath2+'UC/'
GTFS_VC2 = GTFS_filepath2+'VC/'
GTFS_VN2 = GTFS_filepath2+'VN/'
GTFS_VT2 = GTFS_filepath2+'VT/'
GTFS_WC2 = GTFS_filepath2+'WC/'
GTFS_WH2 = GTFS_filepath2+'WH/'

GTFS_baseline = [GTFS_3D,
                    GTFS_AC,
                    GTFS_AM,
                    GTFS_AY,
                    GTFS_BA,
                    GTFS_Caltrain,
                    GTFS_CC,
                    GTFS_CE,
                    GTFS_CM,
                    GTFS_CT,
                    GTFS_DE,
                    GTFS_EM,
                    GTFS_GG,
                    GTFS_HF,
                    GTFS_MA,
                    GTFS_PE,
                    GTFS_RV,
                    GTFS_SB,
                    GTFS_SC,
                    GTFS_SF,
                    GTFS_SM,
                    GTFS_SO,
                    GTFS_SR,
                    GTFS_ST,
                    GTFS_TD,
                    GTFS_UC,
                    GTFS_VC,
                    GTFS_VN,
                    GTFS_VT,
                    GTFS_WC,
                    GTFS_WH]

GTFS_TR = [GTFS_3D2,
                    GTFS_AC2,
                    GTFS_AM2,
                    GTFS_AY2,
                    GTFS_BA2,
                    GTFS_Caltrain2,
                    GTFS_CC2,
                    GTFS_CE2,
                    GTFS_CM2,
                    GTFS_CT2,
                    GTFS_DE2,
                    GTFS_EM2,
                    GTFS_GG2,
                    GTFS_HF2,
                    GTFS_MA2,
                    GTFS_PE2,
                    GTFS_RV2,
                    GTFS_SB2,
                    GTFS_SC2,
                    GTFS_SF2,
                    GTFS_SM2,
                    GTFS_SO2,
                    GTFS_SR2,
                    GTFS_ST2,
                    GTFS_TD2,
                    GTFS_UC2,
                    GTFS_VC2,
                    GTFS_VN2,
                    GTFS_VT2,
                    GTFS_WC2,
                    GTFS_WH2]
GTFS_filepaths = [GTFS_baseline, GTFS_TR,GTFS_baseline, GTFS_TR,GTFS_baseline, GTFS_TR,GTFS_baseline, GTFS_TR]



nrows = None

/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:


#Ridership route
RR = pd.DataFrame()
#Ridership agency
RA = pd.DataFrame()

for fp, name, GTFS_filepath in zip(dataFilepath_sim,names, GTFS_filepaths):
    print('evaluate ridership')
    #import pathtraversal and person enter vehicles
    PT, PE = processEvents(fp, nrows)
    #filter PE transit trips from PT
    PE = filter_transit(PT, PE)
    #Guess transit agency for each PE
    PE = guess_agency(PE)
    #Guess transit route for each PE
    PE = guess_route(PE, GTFS_filepath)
    #Route Ridership
    rr = PE['route_id'].value_counts()
    sum_agency = 0
    sum_agency_bsva = 0
    sum_route = 0
    for route, count in zip(rr.keys(), rr):
        RR.at[route, name] = count
        sum_route += count
    ra = PE['agency'].value_counts()
    for agency, count in zip(ra.keys(), ra):
        RA.at[agency, name] = count
        sum_agency += count
        if agency in ['BA', 'SF', 'VT', 'AC']:
            RA.at[agency, name+' BA-SF-VT-AC'] = count
            sum_agency_bsva += count
    for route, count in zip(rr.keys(), rr):
        RR.at[route, name+' shares'] = count/sum_route
    ra = PE['agency'].value_counts()
    for agency, count in zip(ra.keys(), ra):
        RA.at[agency, name+' shares'] = count/sum_agency
        if agency in ['BA', 'SF', 'VT', 'AC']:
            RA.at[agency, name+' shares BA-SF-VT-AC'] = count/sum_agency_bsva
    RA.to_csv(output_filepath+RA_output)
    RR.to_csv(output_filepath+RR_output)

for year in years:
    diff = []
    diff_abs = []
    for baseline, future in zip(RA['Baseline'+year],RA['Future'+year]):
        diff.append((future-baseline)/baseline)
        diff_abs.append((future-baseline))
    RA['Diff %'+year] = diff
    RA['Diff'+year] = diff_abs

tot_NTD = 1756364558 + 15283299+5703705+49795740+110802986+7386518+49247910+27027693 + 8437926+50222832 + 2818648
    
RA.at['BA', 'clipper BA-SF-VT-AC 2016 share target'] = 350485/773719
RA.at['BA', 'MTC BA-SF-VT-AC 2016 share target'] = 458900/1564500
RA.at['BA', 'NTD BA-SF-VT-AC 2019 share target'] = (1756364558 + 15283299)/tot_NTD

RA.at['SF', 'clipper BA-SF-VT-AC 2016 share target'] = 293991/773719
RA.at['SF', 'MTC BA-SF-VT-AC 2016 share target'] = 777000/1564500
RA.at['SF', 'NTD BA-SF-VT-AC 2019 share target'] = (5703705+49795740+110802986+7386518+49247910)/tot_NTD

RA.at['VT', 'clipper BA-SF-VT-AC 2016 share target'] = 43950/773719
RA.at['VT', 'MTC BA-SF-VT-AC 2016 share target'] = 146700/1564500
RA.at['VT', 'NTD BA-SF-VT-AC 2019 share target'] = (27027693 + 8437926)/tot_NTD

RA.at['AC', 'clipper BA-SF-VT-AC 2016 share target'] = 85293/773719
RA.at['AC', 'MTC BA-SF-VT-AC 2016 share target'] = 181900/1564500
RA.at['AC', 'NTD BA-SF-VT-AC 2019 share target'] = (50222832 + 2818648)/tot_NTD

RA.at['BA', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 350485
RA.at['BA', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 458900
RA.at['BA', 'NTD BA-SF-VT-AC 2019 ridership target'] = (1756364558 + 15283299)

RA.at['SF', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 293991
RA.at['SF', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 777000
RA.at['SF', 'NTD BA-SF-VT-AC 2019 ridership target'] = (5703705+49795740+110802986+7386518+49247910)

RA.at['VT', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 43950
RA.at['VT', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 146700
RA.at['VT', 'NTD BA-SF-VT-AC 2019 ridership target'] =  (27027693 + 8437926)

RA.at['AC', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 85293
RA.at['AC', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 181900
RA.at['AC', 'NTD BA-SF-VT-AC 2019 ridership target'] = (50222832 + 2818648)

RA.to_csv(output_filepath+RA_output)
RR.to_csv(output_filepath+RR_output)




evaluate ridership
read s3://beam-outputs/pilates-outputs/sfbay-baseline-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_71528/3554254780.py:13: DtypeWarning: Columns (0,13,15,16,17,19,20,21,22,29,35,37,39,41,42,43,44,46,47,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


PathTraversal           355896
PersonEntersVehicle     254363
departure               183651
PersonLeavesVehicle     134821
ModeChoice              101597
actend                  101533
LeavingParkingEvent      79407
actstart                 75088
arrival                  75088
PersonCost               66631
ParkingEvent             60125
TeleportationEvent        9433
ReserveRideHail           2291
Replanning                  64
ChargingPlugInEvent          4
RefuelSessionEvent           4
ChargingPlugOutEvent         4
Name: type, dtype: int64


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_71528/3554254780.py:13: DtypeWarning: Columns (0,17,46,51) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


PathTraversal          375409
PersonEntersVehicle    188683
PersonLeavesVehicle    153238
ModeChoice             119235
actend                 118915
departure              118915
arrival                 92021
actstart                92017
LeavingParkingEvent     82265
PersonCost              73897
ParkingEvent            64421
TeleportationEvent      17778
ReserveRideHail          2959
Replanning                247
Name: type, dtype: int64


...

PathTraversal           393038
PersonLeavesVehicle     182217
PersonEntersVehicle     170639
arrival                 106483
actstart                106478
ModeChoice               98066
departure                97561
actend                   97561
PersonCost               82341
LeavingParkingEvent      75016
ParkingEvent             69745
TeleportationEvent       14126
ReserveRideHail           6134
Replanning                 581
RefuelSessionEvent           5
ChargingPlugOutEvent         5
ChargingPlugInEvent          4
Name: type, dtype: int64


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_71528/3554254780.py:13: DtypeWarning: Columns (3,15,49,51,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


PathTraversal           380366
PersonLeavesVehicle     181078
PersonEntersVehicle     171288
arrival                 108263
actstart                108260
ModeChoice              101534
actend                  100745
departure               100744
PersonCost               81432
LeavingParkingEvent      74421
ParkingEvent             69225
TeleportationEvent       15795
ReserveRideHail           6200
Replanning                 646
ChargingPlugInEvent          1
RefuelSessionEvent           1
ChargingPlugOutEvent         1
Name: type, dtype: int64


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_71528/3554254780.py:13: DtypeWarning: Columns (15,20,41) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


PathTraversal          375085
PersonLeavesVehicle    170049
PersonEntersVehicle    127710
actstart               100308
arrival                100297
PersonCost              71523
ModeChoice              67903
departure               67492
actend                  67491
ParkingEvent            63317
LeavingParkingEvent     61207
TeleportationEvent       9341
ReserveRideHail          3781
Replanning                511
Name: type, dtype: int64
         vehicle    time
173810    620727   18007
174050    740481   18021
174057    229418   18050
174068     29974   18064
174079    133521   18003
...          ...     ...
35785469   39198  182176
35785836  528903  205746
35785913  528903  209876
35785932  202477  211277
35786008   92528  215806

[2062981 rows x 2 columns]
car          2770165
walk         1958263
bus          1341993
car_hov2      404751
car_hov3      286217
tram           54924
bike           34530
cable_car      17836
subway         15118
rail            1947
ferry            

In [3]:


RA


,Baseline2018,Baseline2018 BA-SF-VT-AC,Baseline2018 shares,Baseline2018 shares BA-SF-VT-AC,Future2018,Future2018 BA-SF-VT-AC,Future2018 shares,Future2018 shares BA-SF-VT-AC,Diff %2018,Diff2018,clipper BA-SF-VT-AC 2016 share target,MTC BA-SF-VT-AC 2016 share target,NTD BA-SF-VT-AC 2019 share target,clipper BA-SF-VT-AC 2020 Jan av ridership target,MTC BA-SF-VT-AC 2016 av ridership target,NTD BA-SF-VT-AC 2019 ridership target
SF,96467.0,96467.0,0.325643,0.404498,97766.0,97766.0,0.328833,0.408297,0.013466,1299.0,0.379971,0.496644,0.107022,293991.0,777000.0,2.229369e+08
AC,56183.0,56183.0,0.189657,0.235582,56226.0,56226.0,0.189114,0.234815,0.000765,43.0,0.110238,0.116267,0.025463,85293.0,181900.0,5.304148e+07
VT,52724.0,52724.0,0.177980,0.221078,52508.0,52508.0,0.176609,0.219288,-0.004097,-216.0,0.056804,0.093768,0.017025,43950.0,146700.0,3.546562e+07
BA,33112.0,33112.0,0.111776,0.138843,32948.0,32948.0,0.110820,0.137600,-0.004953,-164.0,0.452987,0.293321,0.850490,350485.0,458900.0,1.771648e+09
ST,16540.0,NaN,0.055834,NaN,16209.0,NaN,0.054518,NaN,-0.020012,-331.0,NaN,NaN,NaN,NaN,NaN,NaN
GG,7370.0,NaN,0.024879,NaN,7319.0,NaN,0.024617,NaN,-0.006920,-51.0,NaN,NaN,NaN,NaN,NaN,NaN
CC,5693.0,NaN,0.019218,NaN,5995.0,NaN,0.020164,NaN,0.053048,302.0,NaN,NaN,NaN,NaN,NaN,NaN
WH,3007.0,NaN,0.010151,NaN,3050.0,NaN,0.010259,NaN,0.014300,43.0,NaN,NaN,NaN,NaN,NaN,NaN
3D,2918.0,NaN,0.009850,NaN,3033.0,NaN,0.010201,NaN,0.039411,115.0,NaN,NaN,NaN,NaN,NaN,NaN
SR,2871.0,NaN,0.009692,NaN,2842.0,NaN,0.009559,NaN,-0.010101,-29.0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
RR[RR.index=='SF:18608']

,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
SF:18608,NaN,NaN,4216.0,0.01418


In [5]:

RR[RR.index.str.contains('AC:1-142')]


,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
AC:1-142,2182.0,0.007366,NaN,NaN


In [6]:

RR[RR.index.str.contains('AC:1T-142')]


,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
AC:1T-142,NaN,NaN,2636.0,0.008866


In [7]:



RR[RR.index.str.contains('CA:')]



,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
CA:12868,1157.0,0.003906,1293.0,0.004349
CA:12869,679.0,0.002292,872.0,0.002933
CA:12867,388.0,0.001310,411.0,0.001382


In [8]:


RR[RR.index.str.contains('SF:1000')]


,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
SF:1000,NaN,NaN,1626.0,0.005469


In [9]:

#The Central Subway Project will improve public transportation in San Francisco by extending the Muni Metro T Third Line through SoMa

#J
RR[RR.index.str.contains('SF:12475')]


,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
SF:12475,2368.0,0.007994,2347.0,0.007894


In [10]:

#K/T
RR[RR.index.str.contains('SF:12476')]


,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
SF:12476,4202.0,0.014185,4210.0,0.01416


In [11]:

#L
RR[RR.index.str.contains('SF:12477')]


,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
SF:12477,3148.0,0.010627,3160.0,0.010629


In [12]:

#M
RR[RR.index.str.contains('SF:12478')]


,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
SF:12478,3147.0,0.010623,3204.0,0.010777


In [13]:

#N
RR[RR.index.str.contains('SF:12479')]



,Baseline2018,Baseline2018 shares,Future2018,Future2018 shares
SF:12479,4310.0,0.014549,4414.0,0.014846


In [14]:
for year in years:
    
    print('total SF light rail baseline'+year,
         (list(RR[RR.index.str.contains('SF:12475')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12476')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12477')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12478')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12479')]['Baseline'+year])[0])
         )
    print('total SF light rail future'+year,
         (list(RR[RR.index.str.contains('SF:12475')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12476')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12477')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12478')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12479')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:1000')]['Future'+year])[0])
         )

total SF light rail baseline2018 17175.0
total SF light rail future2018 18961.0


In [15]:
#Imports
import numpy as np
import pandas as pd
import difflib
import matplotlib.pyplot as plt
import time

#########Analysis of what agents used in the baseline
index_baseline =6 
index_future =7
fp_PtoPTss_baseline = '/Users/cpoliziani/Downloads/personToVehicles.csv.gz'
fp_PtoPTss_future = '/Users/cpoliziani/Downloads/personToVehicles (1).csv.gz'

# print('read PtoPTss baseline',fp_PtoPTss_baseline)
PtoPTss_baseline = pd.read_csv(fp_PtoPTss_baseline)
# print('read PtoPTss TR', fp_PtoPTss_future)
PtoPTss_TR = pd.read_csv(fp_PtoPTss_future)
# print('read plans baseline',dataFilepath_sim[index_baseline][:-13]+'plans.csv.gz')
plans_baseline = pd.read_csv(dataFilepath_sim[index_baseline][:-13]+'plans.csv.gz')
# print('read plans TR', dataFilepath_sim[index_future][:-13]+'plans.csv.gz')
plans_TR = pd.read_csv(dataFilepath_sim[index_future][:-13]+'plans.csv.gz')
# print('read GTFS trips fpr AC, SF and CA')
line_1T_trips_TR = pd.read_csv(GTFS_AC2+'trips.txt')
line_1T_trips_TR = line_1T_trips_TR[line_1T_trips_TR['route_id'].str.contains('1T-142')]
line_CS_trips_TR = pd.read_csv(GTFS_SF2+'trips.txt')
line_CS_trips_TR = line_CS_trips_TR[line_CS_trips_TR['route_id']==1000]
line_CA_trips_TR = pd.read_csv(GTFS_Caltrain2+'trips.txt')
# print('read mode choice')
mode_choice_baseline = pd.read_csv(dataFilepath_sim[index_baseline])
mode_choice_TR = pd.read_csv(dataFilepath_sim[index_future])
mode_choice_baseline = mode_choice_baseline[mode_choice_baseline['type']=='ModeChoice']
mode_choice_TR = mode_choice_TR[mode_choice_TR['type']=='ModeChoice']

/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_71528/700297135.py:15: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  PtoPTss_baseline = pd.read_csv(fp_PtoPTss_baseline)
/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_71528/700297135.py:17: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  PtoPTss_TR = pd.read_csv(fp_PtoPTss_future)


IndexError: list index out of range

In [ ]:
PtoPTss_AC_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('AC', na=False)]
PtoPTss_SF_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('SF', na=False)]
PtoPTss_CA_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('Ca', na=False)]

#Filter SF trips - CS only
is_CS = []
vehicles = list(line_CS_trips_TR['trip_id'])
for vehicle in PtoPTss_SF_TR['vehicleID']:
    if int(vehicle[3:]) in vehicles:
        is_CS.append(True)
    else:
        is_CS.append(False)
print('Filter CS trips [tot, remained]',len(is_CS),sum(is_CS))
PtoPTss_SF_TR['is_CS'] = is_CS
PtoPTss_SF_TR = PtoPTss_SF_TR[PtoPTss_SF_TR['is_CS']==True]


is_AC = []
vehicles = list(line_1T_trips_TR['trip_id'])
for vehicle in PtoPTss_AC_TR['vehicleID']:
    if int(vehicle[3:]) in vehicles:
        is_AC.append(True)
    else:
        is_AC.append(False)
print('Filter 1T trips [tot, remained]',len(is_AC),sum(is_AC))
PtoPTss_AC_TR['is_AC'] = is_AC
PtoPTss_AC_TR = PtoPTss_AC_TR[PtoPTss_AC_TR['is_AC']==True]

In [ ]:
persons_baseline = pd.DataFrame()
i = 0
trips_TR = pd.concat([line_CA_trips_TR['trip_id'],line_CS_trips_TR['trip_id'],line_1T_trips_TR['trip_id']])
##############################
# PtoPTss_TR2=pd.concat([PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR])
# PtoPTss_TR2_dict = PtoPTss.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
#                                                     sum(x.duration)]).to_dict()
# print(len(PtoPTss_CA_TR))
PtoPTss_TR_CA_dict = PtoPTss_CA_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration)]).to_dict()
# print(len(PtoPTss_TR_CA_dict.keys()))
# print(len(PtoPTss_SF_TR))
PtoPTss_TR_SF_dict = PtoPTss_SF_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration)]).to_dict()
# print(len(PtoPTss_TR_SF_dict.keys()))
# print(len(PtoPTss_AC_TR))
PtoPTss_TR_AC_dict = PtoPTss_AC_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration)]).to_dict()
# print(len(PtoPTss_TR_AC_dict.keys()))
# print(len(PtoPTss_baseline))
PtoPTss_baseline_dict = PtoPTss_baseline.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration)]).to_dict()
# print(len(PtoPTss_baseline_dict.keys()))
# print(len(PtoPTss_TR))
PtoPTss_TR_dict = PtoPTss_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                sum(x.duration)]).to_dict()
print(len(PtoPTss_TR_dict.keys()))


#Add logsum
person_trips_baseline = pd.read_csv('/Users/cpoliziani/Downloads/tourTripsMerged_Baseline.csv')
person_trips_baseline['trip_num']=person_trips_baseline['trip_num']*2-1
person_trips_TR = pd.read_csv('/Users/cpoliziani/Downloads/tourTripsMerged_TR.csv')
person_trips_TR['trip_num']=person_trips_TR['trip_num']*2-1

In [ ]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:32610'): 
    gdf.set_crs(epsg = "3310", inplace = True)
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.set_crs(epsg = "32610", inplace = True)
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['ZCTA'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'ZCTA': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
#     df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [ ]:
persons_baseline = pd.DataFrame(columns = ['Person',
                                           'Plan Index',
                                           'Planned Depart Time Baseline',
                                           'Planned Depart Time TR',
                                           'Activity From Baseline',
                                           'Activity To Baseline',
                                           'Activity From TR',
                                           'Activity To TR',
                                           'Project Tried',
                                           'Vehicles Used Baseline',
                                           'Vehicle Types Used Baseline',                                          
                                           'Bus agencies Used Baseline',
                                           'First Bus agency Used Baseline',
                                           'Vehicles Used TR',
                                           'Vehicle Types Used TR',                                          
                                           'Bus agencies Used TR',
                                           'First Bus agency Used TR',
                                           'Switch From',
                                           'Trip Length Baseline',                       
                                           'Trip Duration Baseline',
                                           'Trip Length TR',
                                           'Trip Duration TR',
                                           'Diff Length',
                                           'Diff Duration',
                                           'Planned Mode Baseline',                                         
                                           'Planned Mode TR',                                          
                                           'Chosen Mode Baseline',                                          
                                           'Chosen Mode TR',
                                           'X Activity From TR',
                                           'Y Activity From TR',
                                           'X Activity To TR',
                                           'Y Activity To TR',
                                           'X Activity From Baseline',
                                           'Y Activity From Baseline',
                                           'X Activity To Baseline',
                                           'Y Activity To Baseline',
                                           'Log Sum Baseline',
                                           'Log Sum TR',
                                           'Diff Log Sum',
#                                            'ZIP Departure TR',                                          
#                                            'ZIP Arrival TR',                                          
#                                            'ZIP Departure Baseline',                                          
#                                            'ZIP Arrival Baseline',                                          
                                          ])

PtoPTss_baseline_bus =  PtoPTss_baseline[(PtoPTss_baseline['mode'] =='bus')|
                                         (PtoPTss_baseline['mode'] =='subway')|
                                         (PtoPTss_baseline['mode'] =='tram')|
                                         (PtoPTss_baseline['mode'] =='rail')|
                                         (PtoPTss_baseline['mode'] =='cable_car')|
                                         (PtoPTss_baseline['mode'] =='ferry')]
PtoPTss_TR_bus =  PtoPTss_TR[(PtoPTss_TR['mode'] =='bus')|
                                         (PtoPTss_TR['mode'] =='subway')|
                                         (PtoPTss_TR['mode'] =='tram')|
                                         (PtoPTss_TR['mode'] =='rail')|
                                         (PtoPTss_TR['mode'] =='cable_car')|
                                         (PtoPTss_TR['mode'] =='ferry')]   
print(len(PtoPTss_TR_SF_dict))
print(len(PtoPTss_TR_CA_dict))
print(len(PtoPTss_TR_AC_dict))


i=0      
for row in PtoPTss_TR_SF_dict.keys():
    i+=1
    if i%50==0:
        print(i)
        
    try:
        persons_baseline.at[i,'Person'] = row[0]
    except:
        print('Warning', row, 'person')
    try:
        persons_baseline.at[i,'Plan Index'] = int(row[1])
    except:
        print('Warning', row, 'plan_index')
    try:
        persons_baseline.at[i,'Activity From Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From Baseline')
    try:
        persons_baseline.at[i,'Activity To Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To Baseline')
    try:
        persons_baseline.at[i,'Activity From TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From TR')
    try:
        persons_baseline.at[i,'Activity To TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To TR')  
    try:
        persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_baseline')
    try:
        persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_TR')
    try:
        persons_baseline.at[i,'Project Tried'] = 'SF - Central Subway'
    except:
        print('Warning', row, 'project tried')
    try:
        persons_baseline.at[i,'Log Sum Baseline'] = np.mean(list(person_trips_baseline['mode_choice_logsum_y'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Log Sum Baseline')
    try:
        persons_baseline.at[i,'Log Sum TR'] = np.mean(list(person_trips_TR['mode_choice_logsum_y'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Log Sum TR')
    try:
        persons_baseline.at[i,'Diff Log Sum'] = persons_baseline.at[i,'Log Sum TR']-persons_baseline.at[i,'Log Sum Baseline']
    except:
        print('Warning', row, 'Diff Log Sum')
    try:
        persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used Baseline')
    try:
        persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used TR')
    try:
        persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used Baseline')
    try:
        persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used TR') 
    try:
        persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used Baseline')
    try:
        persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used TR')
    try:
        persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
    except:
        print('Warning', row, 'TR_length')
    try:
        persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
    except:
        print('Warning', row, 'TR_duration')
    try:
        persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
    except:
        print('Warning', row, 'baseline_length')
    try:
        persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
    except:
        print('Warning', row, 'baseline_duration')
    try:
        persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
    except:
        print('Warning', row, 'diff_length')
    try:
        persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
    except:
        print('Warning', row, 'diff_duration')
    try:
        persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_TR')
    try:
        persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_TR')
    try:
        persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_TR')
    try:
        persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_TR')   
    try:
        persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_Baseline')
    try:
        persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_Baseline')
    try:
        persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_Baseline')
    try:
        persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_Baseline')   
    try:
        persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used baseline')
    try:
        persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used TR')
    try:
        persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'Baseline planned mode')
    try:
        persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'TR planned mode')
    try:
        plan_dep_time_baseline = list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']>=plan_dep_time_baseline-1)])[0]
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']>=persons_baseline.at[i,'Planned Depart Time TR']-1)])[0]
    except:
        print('Warning', row, 'mode_choice_TR')
        
####################################
for row in PtoPTss_TR_CA_dict.keys():
    i+=1
    if i%10==0:
        print(i)
        
        
    try:
        persons_baseline.at[i,'Person'] = row[0]
    except:
        print('Warning', row, 'person')
    try:
        persons_baseline.at[i,'Plan Index'] = int(row[1])
    except:
        print('Warning', row, 'plan_index')
    try:
        persons_baseline.at[i,'Activity From Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From Baseline')
    try:
        persons_baseline.at[i,'Activity To Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To Baseline')
    try:
        persons_baseline.at[i,'Activity From TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From TR')
    try:
        persons_baseline.at[i,'Activity To TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To TR')  
    try:
        persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_baseline')
    try:
        persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_TR')
    try:
        persons_baseline.at[i,'Project Tried'] = 'CA - Electrification Project'
    except:
        print('Warning', row, 'project tried')
    try:
        persons_baseline.at[i,'Log Sum Baseline'] = np.mean(list(person_trips_baseline['mode_choice_logsum_y'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Log Sum Baseline')
    try:
        persons_baseline.at[i,'Log Sum TR'] = np.mean(list(person_trips_TR['mode_choice_logsum_y'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Log Sum TR')
    try:
        persons_baseline.at[i,'Diff Log Sum'] = persons_baseline.at[i,'Log Sum TR']-persons_baseline.at[i,'Log Sum Baseline']
    except:
        print('Warning', row, 'Diff Log Sum')
    try:
        persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used Baseline')
    try:
        persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used TR')
    try:
        persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used Baseline')
    try:
        persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used TR')
    try:
        persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used Baseline')
    try:
        persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used TR')  
    try:
        persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
    except:
        print('Warning', row, 'TR_length')
    try:
        persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
    except:
        print('Warning', row, 'TR_duration')
    try:
        persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
    except:
        print('Warning', row, 'baseline_length')
    try:
        persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
    except:
        print('Warning', row, 'baseline_duration')
    try:
        persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
    except:
        print('Warning', row, 'diff_length')
    try:
        persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
    except:
        print('Warning', row, 'diff_duration')
    try:
        persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_TR')
    try:
        persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_TR')
    try:
        persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_TR')
    try:
        persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_TR')   
    try:
        persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_Baseline')
    try:
        persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_Baseline')
    try:
        persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_Baseline')
    try:
        persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_Baseline')   
    try:
        persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used baseline')
    try:
        persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used TR')
    try:
        persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'Baseline planned mode')
    try:
        persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'TR planned mode')
    try:
        plan_dep_time_baseline = list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']>=plan_dep_time_baseline-1)])[0]
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']>=persons_baseline.at[i,'Planned Depart Time TR']-1)])[0]
    except:
        print('Warning', row, 'mode_choice_TR')
        
####################################
for row in PtoPTss_TR_AC_dict.keys():
    i+=1
    if i%10==0:
        print(i) 
        
        
    try:
        persons_baseline.at[i,'Person'] = row[0]
    except:
        print('Warning', row, 'person')
    try:
        persons_baseline.at[i,'Plan Index'] = int(row[1])
    except:
        print('Warning', row, 'plan_index')    
    try:
        persons_baseline.at[i,'Activity From Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From Baseline')
    try:
        persons_baseline.at[i,'Activity To Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To Baseline')
    try:
        persons_baseline.at[i,'Activity From TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From TR')
    try:
        persons_baseline.at[i,'Activity To TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To TR')   
    try:
        persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_baseline')
    try:
        persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_TR')
    try:
        persons_baseline.at[i,'Project Tried'] = 'AC - 1TEMPO'
    except:
        print('Warning', row, 'project tried')
    try:
        persons_baseline.at[i,'Log Sum Baseline'] = np.mean(list(person_trips_baseline['mode_choice_logsum_y'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Log Sum Baseline')
    try:
        persons_baseline.at[i,'Log Sum TR'] = np.mean(list(person_trips_TR['mode_choice_logsum_y'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Log Sum TR')
    try:
        persons_baseline.at[i,'Diff Log Sum'] = persons_baseline.at[i,'Log Sum TR']-persons_baseline.at[i,'Log Sum Baseline']
    except:
        print('Warning', row, 'Diff Log Sum')
    try:
        persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used Baseline')
    try:
        persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used TR')    
    try:
        persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used Baseline')
    try:
        persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used TR')
    try:
        persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used Baseline')
    try:
        persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used TR')
    try:
        persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
    except:
        print('Warning', row, 'TR_length')
    try:
        persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
    except:
        print('Warning', row, 'TR_duration')
    try:
        persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
    except:
        print('Warning', row, 'baseline_length')
    try:
        persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
    except:
        print('Warning', row, 'baseline_duration')
    try:
        persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
    except:
        print('Warning', row, 'diff_length')
    try:
        persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
    except:
        print('Warning', row, 'diff_duration')
    try:
        persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_TR')
    try:
        persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_TR')
    try:
        persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_TR')
    try:
        persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_TR')   
    try:
        persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_Baseline')
    try:
        persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_Baseline')
    try:
        persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_Baseline')
    try:
        persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_Baseline')   
    try:
        persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used baseline')
    try:
        persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used TR')
    try:
        persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'Baseline planned mode')
    try:
        persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'TR planned mode')
    try:
        plan_dep_time_baseline = list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']>=plan_dep_time_baseline-1)])[0]
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']>=persons_baseline.at[i,'Planned Depart Time TR']-1)])[0]
    except:
        print('Warning', row, 'mode_choice_TR')

        
zipcode = gpd.read_file('/Users/cpoliziani/Downloads/TAZ to ZIP/ZCTA2010.shp')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity From TR', 'Y Activity From TR', 'ZIP Departure TR')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity To TR', 'Y Activity To TR', 'ZIP Arrival TR')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity From Baseline', 'Y Activity From Baseline', 'ZIP Departure Baseline')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity To Baseline', 'Y Activity To Baseline', 'ZIP Arrival Baseline')


#Add Switch From Column
switch_type = []
for agencies_baseline, project in zip(persons_baseline['Bus agencies Used Baseline'],persons_baseline['Project Tried']):
    if len(agencies_baseline) == 0:
        switch_type.append('Switch from another mode')
    elif project == 'SF - Central Subway' and 'SF' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'AC - 1TEMPO' and 'AC' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'CA - Electrification Project' and 'ca' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    else:
        switch_type.append('Switch from another transit agency')

persons_baseline['Switch From'] = switch_type

persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')



In [ ]:
switch_type = []
for agencies_baseline, project in zip(persons_baseline['Bus agencies Used Baseline'],persons_baseline['Project Tried']):
    if len(agencies_baseline) == 0:
        switch_type.append('Switch from another mode')
    elif project == 'SF - Central Subway' and 'SF' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'AC - 1TEMPO' and 'AC' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'CA - Electrification Project' and 'ca' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    else:
        switch_type.append('Switch from another transit agency')

persons_baseline['Switch From'] = switch_type

persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')


In [ ]:
person_trips_baseline = pd.read_csv('/Users/cpoliziani/Downloads/tourTripsMerged_Baseline.csv')


In [ ]:
person_trips_baseline['trip_num'].value_counts()

In [ ]:
person_trips_baseline

In [ ]:
persons_baseline

In [ ]:
events = pd.read_csv('s3://beam-outputs/output/newyork/nyc-2-ridehail-200k__2022-09-29_11-44-14_esl/ITERS/it.0/0.events.csv.gz')




In [ ]:
veh = events[['vehicle','type']].dropna()
veh = veh[(veh['vehicle'].str.contains('rideHail'))&(veh['type']=='PathTraversal')]
vei = []
for ve in veh['vehicle']:
    vei.append(ve[-10:])

In [ ]:
np.unique(vei, return_counts=True)

In [ ]:
pd.read_csv('/Users/cpoliziani/Downloads/0.skimsRidehail (1).csv.gz')

In [ ]:

# for row in PtoPTss_TR_CA_dict.keys():
#     i+=1
#     if i%1000==0:
#         print(i)
#     try:
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'CA - Electrification Project'
#         persons_baseline.at[i,'TR_length'] = PtoPTss_TR_dict[row][0]
#         persons_baseline.at[i,'TR_duration'] = PtoPTss_TR_dict[row][1]
#         persons_baseline.at[i,'baseline_length'] = PtoPTss_baseline_dict[row][0]
#         persons_baseline.at[i,'baseline_duration'] = PtoPTss_baseline_dict[row][1]
#         persons_baseline.at[i,'diff_length'] = persons_baseline.at[i,'TR_length']-persons_baseline.at[i,'baseline_length']
#         persons_baseline.at[i,'diff_duration'] = persons_baseline.at[i,'TR_duration']-persons_baseline.at[i,'baseline_duration']
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#     except:
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'CA - Electrification Project'
#         print('WARNING:',row[0], row[1]) 


# for row in PtoPTss_TR_AC_dict.keys():
#     i+=1
#     if i%1000==0:
#         print(i)
#     try:
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'AC - 1 Tempo BRT Line'
#         persons_baseline.at[i,'TR_length'] = PtoPTss_TR_dict[row][0]
#         persons_baseline.at[i,'TR_duration'] = PtoPTss_TR_dict[row][1]
#         persons_baseline.at[i,'baseline_length'] = PtoPTss_baseline_dict[row][0]
#         persons_baseline.at[i,'baseline_duration'] = PtoPTss_baseline_dict[row][1]
#         persons_baseline.at[i,'diff_length'] = persons_baseline.at[i,'TR_length']-persons_baseline.at[i,'baseline_length']
#         persons_baseline.at[i,'diff_duration'] = persons_baseline.at[i,'TR_duration']-persons_baseline.at[i,'baseline_duration']
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#     except:
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'AC - 1 Tempo BRT Line'
#         print('WARNING:',row[0], row[1])




##############
# trips_TR = pd.concat([line_CA_trips_TR['trip_id'],line_CS_trips_TR['trip_id'],line_1T_trips_TR['trip_id']])
# for vehicle, person, plan_index, vehicle2 in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'],PtoPTss['vehicle2'] ):
#     if int(vehicle.split(':')[1]) in list(trips_TR):
#         i+=1
#         if i%1000==0:
#             print(i)
#         try:

#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
#             persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'

#             persons_baseline.at[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
#             persons_baseline.at[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
#             persons_baseline.at[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
#             persons_baseline.at[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
#             persons_baseline.at[i,'diff_speed'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
#         except:
#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'

#             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)
#     print('Warning')




# persons_baseline = pd.DataFrame(['person','plan_index','mode_baseline','vehicle_baseline','agency_baseline',
#                                  'x_activity_TR','y_activity_TR','vehicle_now'])
# i = 0
# PtoPTss=pd.concat(PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR)
# for vehicle, person, plan_index in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'] ):
#     if int(vehicle.split(':')[1]) in list(line_1T_trips_TR):
#         i+=1
#         persons_baseline.loc[i,'person'] = person
#         persons_baseline.loc[i,'plan_index'] = plan_index
#         persons_baseline.loc[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
#         persons_baseline.loc[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#         persons_baseline.loc[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#         persons_baseline.loc[i,'vehicle_now'] = '1T'
#         try:
#             persons_baseline.loc[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
#             persons_baseline.loc[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
#             persons_baseline.loc[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
#             persons_baseline.loc[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
#             persons_baseline.loc[i,'speed_length'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
#         except:
#             persons_baseline.loc[i,'vehicle_baseline'] = None
#             persons_baseline.loc[i,'agency_baseline'] = None
#             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)


# persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')

# PtoPTss_baseline


# persons_baseline = pd.DataFrame()
# i = 0
# PtoPTss=pd.concat([PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR])
# for vehicle, person, plan_index, vehicle2 in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'],PtoPTss['vehicle2'] ):
#     if int(vehicle.split(':')[1]) in list(line_1T_trips_TR):
#         i+=1

#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
#             persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'
#         try:

#             persons_baseline.at[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
#             persons_baseline.at[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
#             persons_baseline.at[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
#             persons_baseline.at[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
#             persons_baseline.at[i,'diff_speed'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
#         except:
#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'

#             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)


# persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')


In [ ]:
LIRR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2020LIRR.csv')
LIRR_capacities_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2020LIRR.csv')
LIRR_capacities_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2021LIRR.csv')
LIRR_capacities_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/jan2022LIRR.csv')
LIRR_capacities_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2022LIRR.csv')

MNR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2020MNR.csv')
MNR_capacities_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2020MNR.csv')
MNR_capacities_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2021MNR.csv')
MNR_capacities_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/jan2022MNR.csv')
MNR_capacities_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2022MNR.csv')

# LIRR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/LIRR_capacities.csv')
# MNR_capacities = MNR_capacities.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
# LIRR_capacities = LIRR_capacities.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()


capacities_data = [LIRR_capacities_apr2020,
        LIRR_capacities_aug2020,
        LIRR_capacities_aug2021,
        LIRR_capacities_jan2022,
        LIRR_capacities_may2022,
        MNR_capacities_apr2020,
        MNR_capacities_aug2020,
        MNR_capacities_aug2021,
        MNR_capacities_jan2022,
        MNR_capacities_may2022,
        ]

for capacity_data in capacities_data:
    trains = []
    for train in capacity_data['Train']:
        try:
            trains.append(int(train))
        except:
            trains.append(train)
    capacity_data['Train'] = trains
    


LIRR_capacities_apr2020 = LIRR_capacities_apr2020.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_aug2020 = LIRR_capacities_aug2020.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_aug2021 = LIRR_capacities_aug2021.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_jan2022 = LIRR_capacities_jan2022.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_may2022 = LIRR_capacities_may2022.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()

MNR_capacities_apr2020 = MNR_capacities_apr2020.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_aug2020 = MNR_capacities_aug2020.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_aug2021 = MNR_capacities_aug2021.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_jan2022 = MNR_capacities_jan2022.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_may2022 = MNR_capacities_may2022.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()


LIRR_trips_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Long_Island_Rail_20200318/trips.txt')
LIRR_trips_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Long_Island_Rail_20200629/trips.txt')
LIRR_trips_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Long_Island_Rail_20210726/trips.txt')
LIRR_trips_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Long_Island_Rail_20211216/trips.txt')
LIRR_trips_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Long_Island_Rail_20220430/trips.txt')

LIRR_trips_apr2020['agency_id']='LI'
LIRR_trips_aug2020['agency_id']='LI'
LIRR_trips_aug2021['agency_id']='LI'
LIRR_trips_jan2022['agency_id']='LI'
LIRR_trips_may2022['agency_id']='LI'

MNR_trips_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Metro-North_Railroad_20200325/trips.txt')
MNR_trips_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Metro-North_Railroad_20200731/trips.txt')
MNR_trips_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Metro-North_Railroad_20210721/trips.txt')
MNR_trips_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Metro-North_Railroad_20211222/trips.txt')
MNR_trips_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Metro-North_Railroad_20220429/trips.txt')

MNR_trips_apr2020['agency_id']='1'
MNR_trips_aug2020['agency_id']='1'
MNR_trips_aug2021['agency_id']='1'
MNR_trips_jan2022['agency_id']='1'
MNR_trips_may2022['agency_id']='1'

# MNR_routes_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Metro-North_Railroad_20200325/routes.txt')
# MNR_routes_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Metro-North_Railroad_20200731/routes.txt')
# MNR_routes_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Metro-North_Railroad_20210721/routes.txt')
# MNR_routes_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Metro-North_Railroad_20211222/routes.txt')
# MNR_routes_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Metro-North_Railroad_20220429/routes.txt')

# trips_data_MNR = [MNR_trips_apr2020,
#         MNR_trips_aug2020,
#         MNR_trips_aug2021,
#         MNR_trips_jan2022,
#         MNR_trips_may2022,
#         ]

# routes_data_MNR = [MNR_routes_apr2020,
#         MNR_routes_aug2020,
#         MNR_routes_aug2021,
#         MNR_routes_jan2022,
#         MNR_routes_may2022,
#         ]

# for trip_data_MNR, route_data_MNR, i  in zip(trips_data_MNR,routes_data_MNR, range(len(trips_data))):
#     agencies =[]
#     for route in trip_data_MNR['route_id']:
#         agencies.append(list(route_data_MNR['agency_id'][route_data_MNR['route_id']==route])[0])
#     trip_data_MNR['agency_id']=agencies


# MNR_trips_apr2020 = trips_data_MNR[0]
# MNR_trips_aug2020 = trips_data_MNR[1]
# MNR_trips_aug2021 = trips_data_MNR[2]
# MNR_trips_jan2022 = trips_data_MNR[3]
# MNR_trips_may2022 = trips_data_MNR[4]

trips_data = [LIRR_trips_apr2020,
        LIRR_trips_aug2020,
        LIRR_trips_aug2021,
        LIRR_trips_jan2022,
        LIRR_trips_may2022,
        MNR_trips_apr2020,
        MNR_trips_aug2020,
        MNR_trips_aug2021,
        MNR_trips_jan2022,
        MNR_trips_may2022,
        ]

capacities_data = [LIRR_capacities_apr2020,
        LIRR_capacities_aug2020,
        LIRR_capacities_aug2021,
        LIRR_capacities_jan2022,
        LIRR_capacities_may2022,
        MNR_capacities_apr2020,
        MNR_capacities_aug2020,
        MNR_capacities_aug2021,
        MNR_capacities_jan2022,
        MNR_capacities_may2022,
        ]

for trip_data, capacity_data in zip(trips_data, capacities_data):

    capacities = []
    wrong_ids = []
    print('number of trip', len(trip_data),'number of capacities', len(capacity_data))
    print('number of unique trip', len(np.unique(trip_data['trip_short_name'])))
    for trip_short_name in trip_data['trip_short_name']:
        try:
            capacities.append(capacity_data[trip_short_name][0][0])
        except:
            try:
                capacities.append(capacity_data[float(trip_short_name)][0][0])
            except:
                wrong_ids.append(trip_short_name)
                capacities.append(np.nan)
    trip_data['capacity'] = capacities
    print('Warning!! Not found',len(wrong_ids))
#     print(trip_data)
    print('######################')



In [ ]:
for trip_data in trips_data:
    max_cap = max(list(trip_data['capacity'].dropna()))
    print(max_cap)
    trip_data['capacity'] = trip_data['capacity'].fillna(max_cap)


In [ ]:


trips_data_apr2020 = [trips_data[0],trips_data[5]]
trips_data_aug2020 = [trips_data[1],trips_data[6]]
trips_data_aug2021 = [trips_data[2],trips_data[7]]
trips_data_jan2022 = [trips_data[3],trips_data[8]]
trips_data_may2022 = [trips_data[4],trips_data[9]]

transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_apr2020,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20200318:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20200325:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_apr2020.csv')


transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_aug2020,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20200629:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20200731:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_aug2020.csv')



transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_aug2021,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20210726:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20210721:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_aug2021.csv')



transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_jan2022,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20211216:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20211222:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_jan2022.csv')



transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                            'capacity',
                                           'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_may2022,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20220430:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20220429:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_may2022.csv')



In [ ]:
trips_data_apr2020[1]

In [ ]:
pd.read_csv('s3://beam-outputs/output/newyork/new-york-jan2022-0-of-10__2022-09-21_17-07-49_qdx/ITERS/it.10/10.events.csv.gz', nrows = 90000)




In [ ]:
# #Imports
# import numpy as np
# import pandas as pd
# import geopandas as gpd
# import difflib
# import matplotlib.pyplot as plt
# import time

# def  processEvents(dataFilepath_sim, nrows):
#     PTs = []
#     PEVs = []
#     print('read', dataFilepath_sim)
#     for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):
#         if sum((chunk['type'] == 'PathTraversal')) > 0:
#             chunk['vehicle'] = chunk['vehicle'].astype(str)
#             PT = chunk.loc[(chunk['type'] == 'PathTraversal') & (chunk['length'] > 0)].dropna(how='all', axis=1)
#             PT['departureTime'] = PT['departureTime'].astype(int)
#             PT['arrivalTime'] = PT['arrivalTime'].astype(int)

#             PTs.append(PT[['driver', 'vehicle', 'mode', 'length', 'startX', 'startY', 'endX', 'endY', 'vehicleType',
#                            'arrivalTime', 'departureTime', 'primaryFuel', 'primaryFuelType', 'secondaryFuel',
#                            'secondaryFuelType', 'numPassengers', 'riders','time']])
#             print(chunk.type.value_counts())
#             PEV = chunk.loc[(chunk.type == "PersonEntersVehicle") &
#                             ~(chunk['person'].apply(str).str.contains('Agent').fillna(False)) &
#                             ~(chunk['vehicle'].str.contains('body').fillna(False)), :].dropna(how='all', axis=1)
#             if ~PEV.empty:
#                 PEV['time'] = PEV['time'].astype(int)
#                 PEVs.append(PEV)

#     Events_file_sim_PT = pd.concat(PTs)[['vehicle','time','endY','endX','startY','startX','mode']]
#     Events_file_sim_PE = pd.concat(PEVs)[['vehicle','time',]]
#     print(Events_file_sim_PE)

#     return Events_file_sim_PT, Events_file_sim_PE



# #Filter transit trips
# def filter_transit(Events_file_sim_PT,Events_file_sim_PE):
#     print(Events_file_sim_PT['mode'].value_counts())
#     Events_file_sim_PT = Events_file_sim_PT[(Events_file_sim_PT['mode'] == 'bus')|
#                                             (Events_file_sim_PT['mode'] == 'tram')|
#                                             (Events_file_sim_PT['mode'] == 'subway')|
#                                             (Events_file_sim_PT['mode'] == 'cable_car')|
#                                             (Events_file_sim_PT['mode'] == 'ferry')|
#                                             (Events_file_sim_PT['mode'] == 'rail')]
#     print(Events_file_sim_PT['mode'].value_counts())
#     Events_file_sim_PE = Events_file_sim_PE[Events_file_sim_PE['vehicle'].isin(Events_file_sim_PT['vehicle'])]
    
#     return Events_file_sim_PE

# def guess_agency(Events_file_sim_PE):

#     agencies = []
#     for vehicleID in Events_file_sim_PE['vehicle']:
#         agency = vehicleID.split(':')[0]
#         if agency == 'petalumatransit-petaluma-ca-us':
#             agencies.append('PE')
#         elif agency == 'westcat-ca-us':
#             agencies.append('WC')
#         elif agency == 'caltrain-ca-us':
#             agencies.append('CA')
#         elif agency == 'riovista-ca-us':
#             agencies.append('RV')
#         elif agency == 'unioncity-ca-us':
#             agencies.append('UC')
#         else:
#             if len(agency) == 2:
#                 agencies.append(agency)
#             elif agency == 'Caltrain':
#                 agencies.append('CA')
#             else:
#                 print('Warning, this agency is not recognized:', agency)
#     Events_file_sim_PE['agency'] = agencies

#     print(np.unique(agencies))
    
#     return Events_file_sim_PE

# def guess_route(Events_file_sim_PE, GTFS_filepaths):

#     GTFS_trip_files = {}

#     for GTFS_filepath, GTFS in zip(GTFS_filepaths,GTFSs):
#         GTFS_trip_files[GTFS] = pd.read_csv(GTFS_filepath+'trips.txt')

#     route_ids = []
#     total_routes = len(Events_file_sim_PE['vehicle'])
#     i = 0
#     time_start = time.time()
#     for vehicle, agency in zip(Events_file_sim_PE['vehicle'],Events_file_sim_PE['agency']):
#         i+=1
#         if i%10000 ==0:
#             print(i,'/',total_routes,'. Time = ', time.time()-time_start, '. Estimated remaining time:', (time.time()-time_start)/i*total_routes-(time.time()-time_start))
#         if agency == 'SM':
#             route_ids.append(agency+':'+str(list(GTFS_trip_files[agency]['route_id'][
#                 GTFS_trip_files[agency]['trip_id'].astype(str)==
#                      str(vehicle).split(':')[1]+'|'
#                      +str(vehicle).split(':')[2]+':'
#                      +str(vehicle).split(':')[3]+'|'
#                      +str(vehicle).split(':')[4]+':'
#                      +str(vehicle).split(':')[5]+':'
#                      +str(vehicle).split(':')[6]])[0]))
#         else:
# #             print(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id'].astype(str)==str(vehicle.split(':')[1])])[0])
#             try:
#                 route_ids.append(agency+':'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id'].astype(str)==str(vehicle.split(':')[1])])[0]))
#             except:
#                 print('Warning, trip non found for vehicle', vehicle)
#                 route_ids.append('tripID not found')
#     #     elif agency == 'GG':
#     #         route_ids.append('GG:'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id']==vehicle.split(':')[1]])[0]))
#     #     elif agency == 'SF':
#     #         route_ids.append('SF:'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id']==vehicle.split(':')[1]])[0]))
#     Events_file_sim_PE['route_id'] = route_ids
    
#     return Events_file_sim_PE












# dataFilepath_sim = [
# #                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
# #                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
# #                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
# #                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
# #                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
# #                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
# #                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
# #                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#             ]

                    
# years = ['2018','2019','2020','2021']

# names = []
# for year in years:
#     names.append('Baseline'+year)
#     names.append('Future'+year)

# output_filepath = '/Users/cpoliziani/Downloads/Transit Rich/Results/'

# GTFS_filepath = '/Users/cpoliziani/Downloads/Data/TR/GTFS/r5-simple-no-local/'
# GTFS_filepath2 = '/Users/cpoliziani/Downloads/Data/TR/GTFS/r5-simple-no-local-TR/'

# RA_output = 'agency_ridershipNew3.csv'
# RR_output = 'route_ridershipNew3.csv'

# GTFSs = ['3D','AC','AM','AY','BA','CA','CC','CE','CM','CT','DE','EM','GG','HF','MA','PE','RV', 'SB',
#         'SC','SF','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH']

# GTFS_3D = GTFS_filepath+'3D/'
# GTFS_AC = GTFS_filepath+'AC/'
# GTFS_AM = GTFS_filepath+'AM/'
# GTFS_AY = GTFS_filepath+'AY/'
# GTFS_BA = GTFS_filepath+'BA/'
# GTFS_Caltrain = GTFS_filepath+'Caltrain/'
# GTFS_CC = GTFS_filepath+'CC/'
# GTFS_CE = GTFS_filepath+'CE/'
# GTFS_CM = GTFS_filepath+'CM/'
# GTFS_CT = GTFS_filepath+'CT/'
# GTFS_DE = GTFS_filepath+'DE/'
# GTFS_EM = GTFS_filepath+'EM/'
# GTFS_GG = GTFS_filepath+'GG/'
# GTFS_HF = GTFS_filepath+'HF/'
# GTFS_MA = GTFS_filepath+'MA/'
# GTFS_PE = GTFS_filepath+'PE/'
# GTFS_RV = GTFS_filepath+'RV/'
# GTFS_SB = GTFS_filepath+'SB/'
# GTFS_SC = GTFS_filepath+'SC/'
# GTFS_SF = GTFS_filepath+'SF/'
# GTFS_SM = GTFS_filepath+'SM/'
# GTFS_SO = GTFS_filepath+'SO/'
# GTFS_SR = GTFS_filepath+'SR/'
# GTFS_ST = GTFS_filepath+'ST/'
# GTFS_TD = GTFS_filepath+'TD/'
# GTFS_UC = GTFS_filepath+'UC/'
# GTFS_VC = GTFS_filepath+'VC/'
# GTFS_VN = GTFS_filepath+'VN/'
# GTFS_VT = GTFS_filepath+'VT/'
# GTFS_WC = GTFS_filepath+'WC/'
# GTFS_WH = GTFS_filepath+'WH/'


# GTFS_3D2 = GTFS_filepath2+'3D/'
# GTFS_AC2 = GTFS_filepath2+'AC/'
# GTFS_AM2 = GTFS_filepath2+'AM/'
# GTFS_AY2 = GTFS_filepath2+'AY/'
# GTFS_BA2 = GTFS_filepath2+'BA/'
# GTFS_Caltrain2 = GTFS_filepath2+'Caltrain/'
# GTFS_CC2 = GTFS_filepath2+'CC/'
# GTFS_CE2 = GTFS_filepath2+'CE/'
# GTFS_CM2 = GTFS_filepath2+'CM/'
# GTFS_CT2 = GTFS_filepath2+'CT/'
# GTFS_DE2 = GTFS_filepath2+'DE/'
# GTFS_EM2 = GTFS_filepath2+'EM/'
# GTFS_GG2 = GTFS_filepath2+'GG/'
# GTFS_HF2 = GTFS_filepath2+'HF/'
# GTFS_MA2 = GTFS_filepath2+'MA/'
# GTFS_PE2 = GTFS_filepath2+'PE/'
# GTFS_RV2 = GTFS_filepath2+'RV/'
# GTFS_SB2 = GTFS_filepath2+'SB/'
# GTFS_SC2 = GTFS_filepath2+'SC/'
# GTFS_SF2 = GTFS_filepath2+'SF/'
# GTFS_SM2 = GTFS_filepath2+'SM/'
# GTFS_SO2 = GTFS_filepath2+'SO/'
# GTFS_SR2 = GTFS_filepath2+'SR/'
# GTFS_ST2 = GTFS_filepath2+'ST/'
# GTFS_TD2 = GTFS_filepath2+'TD/'
# GTFS_UC2 = GTFS_filepath2+'UC/'
# GTFS_VC2 = GTFS_filepath2+'VC/'
# GTFS_VN2 = GTFS_filepath2+'VN/'
# GTFS_VT2 = GTFS_filepath2+'VT/'
# GTFS_WC2 = GTFS_filepath2+'WC/'
# GTFS_WH2 = GTFS_filepath2+'WH/'

# GTFS_baseline = [GTFS_3D,
#                     GTFS_AC,
#                     GTFS_AM,
#                     GTFS_AY,
#                     GTFS_BA,
#                     GTFS_Caltrain,
#                     GTFS_CC,
#                     GTFS_CE,
#                     GTFS_CM,
#                     GTFS_CT,
#                     GTFS_DE,
#                     GTFS_EM,
#                     GTFS_GG,
#                     GTFS_HF,
#                     GTFS_MA,
#                     GTFS_PE,
#                     GTFS_RV,
#                     GTFS_SB,
#                     GTFS_SC,
#                     GTFS_SF,
#                     GTFS_SM,
#                     GTFS_SO,
#                     GTFS_SR,
#                     GTFS_ST,
#                     GTFS_TD,
#                     GTFS_UC,
#                     GTFS_VC,
#                     GTFS_VN,
#                     GTFS_VT,
#                     GTFS_WC,
#                     GTFS_WH]

# GTFS_TR = [GTFS_3D2,
#                     GTFS_AC2,
#                     GTFS_AM2,
#                     GTFS_AY2,
#                     GTFS_BA2,
#                     GTFS_Caltrain2,
#                     GTFS_CC2,
#                     GTFS_CE2,
#                     GTFS_CM2,
#                     GTFS_CT2,
#                     GTFS_DE2,
#                     GTFS_EM2,
#                     GTFS_GG2,
#                     GTFS_HF2,
#                     GTFS_MA2,
#                     GTFS_PE2,
#                     GTFS_RV2,
#                     GTFS_SB2,
#                     GTFS_SC2,
#                     GTFS_SF2,
#                     GTFS_SM2,
#                     GTFS_SO2,
#                     GTFS_SR2,
#                     GTFS_ST2,
#                     GTFS_TD2,
#                     GTFS_UC2,
#                     GTFS_VC2,
#                     GTFS_VN2,
#                     GTFS_VT2,
#                     GTFS_WC2,
#                     GTFS_WH2]
# GTFS_filepaths = [GTFS_baseline, GTFS_TR,GTFS_baseline, GTFS_TR,GTFS_baseline, GTFS_TR,GTFS_baseline, GTFS_TR]



# nrows = None



# #Ridership route
# RR = pd.DataFrame()
# #Ridership agency
# RA = pd.DataFrame()

# for fp, name, GTFS_filepath in zip(dataFilepath_sim,names, GTFS_filepaths):
#     print('evaluate ridership')
#     #import pathtraversal and person enter vehicles
#     PT, PE = processEvents(fp, nrows)
#     #filter PE transit trips from PT
#     PE = filter_transit(PT, PE)
#     #Guess transit agency for each PE
#     PE = guess_agency(PE)
#     #Guess transit route for each PE
#     PE = guess_route(PE, GTFS_filepath)
#     #Route Ridership
#     rr = PE['route_id'].value_counts()
#     sum_agency = 0
#     sum_agency_bsva = 0
#     sum_route = 0
#     for route, count in zip(rr.keys(), rr):
#         RR.at[route, name] = count
#         sum_route += count
#     ra = PE['agency'].value_counts()
#     for agency, count in zip(ra.keys(), ra):
#         RA.at[agency, name] = count
#         sum_agency += count
#         if agency in ['BA', 'SF', 'VT', 'AC']:
#             RA.at[agency, name+' BA-SF-VT-AC'] = count
#             sum_agency_bsva += count
#     for route, count in zip(rr.keys(), rr):
#         RR.at[route, name+' shares'] = count/sum_route
#     ra = PE['agency'].value_counts()
#     for agency, count in zip(ra.keys(), ra):
#         RA.at[agency, name+' shares'] = count/sum_agency
#         if agency in ['BA', 'SF', 'VT', 'AC']:
#             RA.at[agency, name+' shares BA-SF-VT-AC'] = count/sum_agency_bsva
#     RA.to_csv(output_filepath+RA_output)
#     RR.to_csv(output_filepath+RR_output)

# for year in years:
#     diff = []
#     diff_abs = []
#     for baseline, future in zip(RA['Baseline'+year],RA['Future'+year]):
#         diff.append((future-baseline)/baseline)
#         diff_abs.append((future-baseline))
#     RA['Diff %'+year] = diff
#     RA['Diff'+year] = diff_abs

# tot_NTD = 1756364558 + 15283299+5703705+49795740+110802986+7386518+49247910+27027693 + 8437926+50222832 + 2818648
    
# RA.at['BA', 'clipper BA-SF-VT-AC 2016 share target'] = 350485/773719
# RA.at['BA', 'MTC BA-SF-VT-AC 2016 share target'] = 458900/1564500
# RA.at['BA', 'NTD BA-SF-VT-AC 2019 share target'] = (1756364558 + 15283299)/tot_NTD

# RA.at['SF', 'clipper BA-SF-VT-AC 2016 share target'] = 293991/773719
# RA.at['SF', 'MTC BA-SF-VT-AC 2016 share target'] = 777000/1564500
# RA.at['SF', 'NTD BA-SF-VT-AC 2019 share target'] = (5703705+49795740+110802986+7386518+49247910)/tot_NTD

# RA.at['VT', 'clipper BA-SF-VT-AC 2016 share target'] = 43950/773719
# RA.at['VT', 'MTC BA-SF-VT-AC 2016 share target'] = 146700/1564500
# RA.at['VT', 'NTD BA-SF-VT-AC 2019 share target'] = (27027693 + 8437926)/tot_NTD

# RA.at['AC', 'clipper BA-SF-VT-AC 2016 share target'] = 85293/773719
# RA.at['AC', 'MTC BA-SF-VT-AC 2016 share target'] = 181900/1564500
# RA.at['AC', 'NTD BA-SF-VT-AC 2019 share target'] = (50222832 + 2818648)/tot_NTD

# RA.at['BA', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 350485
# RA.at['BA', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 458900
# RA.at['BA', 'NTD BA-SF-VT-AC 2019 ridership target'] = (1756364558 + 15283299)

# RA.at['SF', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 293991
# RA.at['SF', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 777000
# RA.at['SF', 'NTD BA-SF-VT-AC 2019 ridership target'] = (5703705+49795740+110802986+7386518+49247910)

# RA.at['VT', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 43950
# RA.at['VT', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 146700
# RA.at['VT', 'NTD BA-SF-VT-AC 2019 ridership target'] =  (27027693 + 8437926)

# RA.at['AC', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 85293
# RA.at['AC', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 181900
# RA.at['AC', 'NTD BA-SF-VT-AC 2019 ridership target'] = (50222832 + 2818648)

# RA.to_csv(output_filepath+RA_output)
# RR.to_csv(output_filepath+RR_output)






# RA



# RR[RR.index.str.contains('AC:1-142')]



# RR[RR.index.str.contains('AC:1T-142')]





# RR[RR.index.str.contains('CA:')]





# RR[RR.index.str.contains('SF:1000')]



# #The Central Subway Project will improve public transportation in San Francisco by extending the Muni Metro T Third Line through SoMa

# #J
# RR[RR.index.str.contains('SF:12475')]



# #K/T
# RR[RR.index.str.contains('SF:12476')]



# #L
# RR[RR.index.str.contains('SF:12477')]



# #M
# RR[RR.index.str.contains('SF:12478')]



# #N
# RR[RR.index.str.contains('SF:12479')]



# for year in years:
    
#     print('total SF light rail baseline'+year,
#          (list(RR[RR.index.str.contains('SF:12475')]['Baseline'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12476')]['Baseline'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12477')]['Baseline'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12478')]['Baseline'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12479')]['Baseline'+year])[0])
#          )
#     print('total SF light rail future'+year,
#          (list(RR[RR.index.str.contains('SF:12475')]['Future'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12476')]['Future'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12477')]['Future'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12478')]['Future'+year])[0]+
#           list(RR[RR.index.str.contains('SF:12479')]['Future'+year])[0]+
#           list(RR[RR.index.str.contains('SF:1000')]['Future'+year])[0])
#          )

# #Imports
# import numpy as np
# import pandas as pd
# import difflib
# import matplotlib.pyplot as plt
# import time

# #########Analysis of what agents used in the baseline
# index_baseline =6 
# index_future =7
# fp_PtoPTss_baseline = '/Users/cpoliziani/Downloads/personToVehicles.csv.gz'
# fp_PtoPTss_future = '/Users/cpoliziani/Downloads/personToVehicles (1).csv.gz'

# # print('read PtoPTss baseline',fp_PtoPTss_baseline)
# PtoPTss_baseline = pd.read_csv(fp_PtoPTss_baseline)
# # print('read PtoPTss TR', fp_PtoPTss_future)
# PtoPTss_TR = pd.read_csv(fp_PtoPTss_future)
# # print('read plans baseline',dataFilepath_sim[index_baseline][:-13]+'plans.csv.gz')
# plans_baseline = pd.read_csv(dataFilepath_sim[index_baseline][:-13]+'plans.csv.gz')
# # print('read plans TR', dataFilepath_sim[index_future][:-13]+'plans.csv.gz')
# plans_TR = pd.read_csv(dataFilepath_sim[index_future][:-13]+'plans.csv.gz')
# # print('read GTFS trips fpr AC, SF and CA')
# line_1T_trips_TR = pd.read_csv(GTFS_AC2+'trips.txt')
# line_1T_trips_TR = line_1T_trips_TR[line_1T_trips_TR['route_id'].str.contains('1T-142')]
# line_CS_trips_TR = pd.read_csv(GTFS_SF2+'trips.txt')
# line_CS_trips_TR = line_CS_trips_TR[line_CS_trips_TR['route_id']==1000]
# line_CA_trips_TR = pd.read_csv(GTFS_Caltrain2+'trips.txt')
# # print('read mode choice')
# mode_choice_baseline = pd.read_csv(dataFilepath_sim[index_baseline])
# mode_choice_TR = pd.read_csv(dataFilepath_sim[index_future])
# mode_choice_baseline = mode_choice_baseline[mode_choice_baseline['type']=='ModeChoice']
# mode_choice_TR = mode_choice_TR[mode_choice_TR['type']=='ModeChoice']

# PtoPTss_AC_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('AC', na=False)]
# PtoPTss_SF_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('SF', na=False)]
# PtoPTss_CA_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('Ca', na=False)]

# #Filter SF trips - CS only
# is_CS = []
# vehicles = list(line_CS_trips_TR['trip_id'])
# for vehicle in PtoPTss_SF_TR['vehicleID']:
#     if int(vehicle[3:]) in vehicles:
#         is_CS.append(True)
#     else:
#         is_CS.append(False)
# print('Filter CS trips [tot, remained]',len(is_CS),sum(is_CS))
# PtoPTss_SF_TR['is_CS'] = is_CS
# PtoPTss_SF_TR = PtoPTss_SF_TR[PtoPTss_SF_TR['is_CS']==True]


# is_AC = []
# vehicles = list(line_1T_trips_TR['trip_id'])
# for vehicle in PtoPTss_AC_TR['vehicleID']:
#     if int(vehicle[3:]) in vehicles:
#         is_AC.append(True)
#     else:
#         is_AC.append(False)
# print('Filter 1T trips [tot, remained]',len(is_AC),sum(is_AC))
# PtoPTss_AC_TR['is_AC'] = is_AC
# PtoPTss_AC_TR = PtoPTss_AC_TR[PtoPTss_AC_TR['is_AC']==True]

# persons_baseline = pd.DataFrame()
# i = 0
# trips_TR = pd.concat([line_CA_trips_TR['trip_id'],line_CS_trips_TR['trip_id'],line_1T_trips_TR['trip_id']])
# ##############################
# # PtoPTss_TR2=pd.concat([PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR])
# # PtoPTss_TR2_dict = PtoPTss.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
# #                                                     sum(x.duration)]).to_dict()
# # print(len(PtoPTss_CA_TR))
# PtoPTss_TR_CA_dict = PtoPTss_CA_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
#                                                     sum(x.duration)]).to_dict()
# # print(len(PtoPTss_TR_CA_dict.keys()))
# # print(len(PtoPTss_SF_TR))
# PtoPTss_TR_SF_dict = PtoPTss_SF_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
#                                                     sum(x.duration)]).to_dict()
# # print(len(PtoPTss_TR_SF_dict.keys()))
# # print(len(PtoPTss_AC_TR))
# PtoPTss_TR_AC_dict = PtoPTss_AC_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
#                                                     sum(x.duration)]).to_dict()
# # print(len(PtoPTss_TR_AC_dict.keys()))
# # print(len(PtoPTss_baseline))
# PtoPTss_baseline_dict = PtoPTss_baseline.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
#                                                     sum(x.duration)]).to_dict()
# # print(len(PtoPTss_baseline_dict.keys()))
# # print(len(PtoPTss_TR))
# PtoPTss_TR_dict = PtoPTss_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
#                                                 sum(x.duration)]).to_dict()
# print(len(PtoPTss_TR_dict.keys()))


# #Add logsum
# person_trips_baseline = pd.read_csv('/Users/cpoliziani/Downloads/tourTripsMerged_Baseline.csv')
# person_trips_baseline['trip_num']=person_trips_baseline['trip_num']*2-1
# person_trips_TR = pd.read_csv('/Users/cpoliziani/Downloads/tourTripsMerged_TR.csv')
# person_trips_TR['trip_num']=person_trips_TR['trip_num']*2-1


# def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:32610'): 
#     gdf.set_crs(epsg = "3310", inplace = True)
#     gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
#     gdf_data.set_crs(epsg = "32610", inplace = True)
#     joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
#     gdf_data = gdf_data.merge(joined['ZCTA'], left_index=True, right_index=True, how="left")
#     gdf_data.rename(columns={'ZCTA': idColumn}, inplace=True)
#     df = pd.DataFrame(gdf_data.drop(columns='geometry'))
# #     df.drop(columns=[xcol, ycol], inplace=True)
#     return df.loc[~df.index.duplicated(keep='first'), :]

# persons_baseline = pd.DataFrame(columns = ['Person',
#                                            'Plan Index',
#                                            'Planned Depart Time Baseline',
#                                            'Planned Depart Time TR',
#                                            'Activity From Baseline',
#                                            'Activity To Baseline',
#                                            'Activity From TR',
#                                            'Activity To TR',
#                                            'Project Tried',
#                                            'Vehicles Used Baseline',
#                                            'Vehicle Types Used Baseline',                                          
#                                            'Bus agencies Used Baseline',
#                                            'First Bus agency Used Baseline',
#                                            'Vehicles Used TR',
#                                            'Vehicle Types Used TR',                                          
#                                            'Bus agencies Used TR',
#                                            'First Bus agency Used TR',
#                                            'Switch From',
#                                            'Trip Length Baseline',                       
#                                            'Trip Duration Baseline',
#                                            'Trip Length TR',
#                                            'Trip Duration TR',
#                                            'Diff Length',
#                                            'Diff Duration',
#                                            'Planned Mode Baseline',                                         
#                                            'Planned Mode TR',                                          
#                                            'Chosen Mode Baseline',                                          
#                                            'Chosen Mode TR',
#                                            'X Activity From TR',
#                                            'Y Activity From TR',
#                                            'X Activity To TR',
#                                            'Y Activity To TR',
#                                            'X Activity From Baseline',
#                                            'Y Activity From Baseline',
#                                            'X Activity To Baseline',
#                                            'Y Activity To Baseline',
#                                            'Log Sum Baseline',
#                                            'Log Sum TR',
#                                            'Diff Log Sum TR',
# #                                            'ZIP Departure TR',                                          
# #                                            'ZIP Arrival TR',                                          
# #                                            'ZIP Departure Baseline',                                          
# #                                            'ZIP Arrival Baseline',                                          
#                                           ])

# PtoPTss_baseline_bus =  PtoPTss_baseline[(PtoPTss_baseline['mode'] =='bus')|
#                                          (PtoPTss_baseline['mode'] =='subway')|
#                                          (PtoPTss_baseline['mode'] =='tram')|
#                                          (PtoPTss_baseline['mode'] =='rail')|
#                                          (PtoPTss_baseline['mode'] =='cable_car')|
#                                          (PtoPTss_baseline['mode'] =='ferry')]
# PtoPTss_TR_bus =  PtoPTss_TR[(PtoPTss_TR['mode'] =='bus')|
#                                          (PtoPTss_TR['mode'] =='subway')|
#                                          (PtoPTss_TR['mode'] =='tram')|
#                                          (PtoPTss_TR['mode'] =='rail')|
#                                          (PtoPTss_TR['mode'] =='cable_car')|
#                                          (PtoPTss_TR['mode'] =='ferry')]   
# print(len(PtoPTss_TR_SF_dict))
# print(len(PtoPTss_TR_CA_dict))
# print(len(PtoPTss_TR_AC_dict))


# i=0      
# for row in PtoPTss_TR_SF_dict.keys():
#     i+=1
#     if i%50==0:
#         print(i)
#         break
#     try:
#         persons_baseline.at[i,'Person'] = row[0]
#     except:
#         print('Warning', row, 'person')
#     try:
#         persons_baseline.at[i,'Plan Index'] = int(row[1])
#     except:
#         print('Warning', row, 'plan_index')
#     try:
#         persons_baseline.at[i,'Activity From Baseline'] = int(list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'Activity From Baseline')
#     try:
#         persons_baseline.at[i,'Activity To Baseline'] = int(list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0])
#     except:
#         print('Warning', row, 'Activity To Baseline')
#     try:
#         persons_baseline.at[i,'Activity From TR'] = int(list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'Activity From TR')
#     try:
#         persons_baseline.at[i,'Activity To TR'] = int(list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0])
#     except:
#         print('Warning', row, 'Activity To TR')  
#     try:
#         persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'plan_dep_time_baseline')
#     try:
#         persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'plan_dep_time_TR')
#     try:
#         persons_baseline.at[i,'Project Tried'] = 'SF - Central Subway'
#     except:
#         print('Warning', row, 'project tried')
#     try:
#         persons_baseline.at[i,'Log Sum Baseline'] = np.unique(list(person_trips_baseline['mode_choice_logsum_y'][(person_trips_baseline['person_id']==row[0])&(person_trips_baseline['trip_num']==row[1])]))
#     except:
#         print('Warning', row, 'Log Sum Baseline')
#     try:
#         persons_baseline.at[i,'Log Sum TR'] = np.unique(list(person_TR['mode_choice_logsum_y'][(person_TR['person_id']==row[0])&(person_TR['trip_num']==row[1])]))
#     except:
#         print('Warning', row, 'Log Sum TR')
#     try:
#         persons_baseline.at[i,'Diff Log Sum'] = persons_baseline.at[i,'Log Sum TR']-persons_baseline.at[i,'Log Sum Baseline']
#     except:
#         print('Warning', row, 'Diff Log Sum TR')
#     try:
#         persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicles Used Baseline')
#     try:
#         persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicles Used TR')
#     try:
#         persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Bus agencies Used Baseline')
#     try:
#         persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Bus agencies Used TR') 
#     try:
#         persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
#     except:
#         print('Warning', row, 'First Bus agency Used Baseline')
#     try:
#         persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
#     except:
#         print('Warning', row, 'First Bus agency Used TR')
#     try:
#         persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
#     except:
#         print('Warning', row, 'TR_length')
#     try:
#         persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
#     except:
#         print('Warning', row, 'TR_duration')
#     try:
#         persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
#     except:
#         print('Warning', row, 'baseline_length')
#     try:
#         persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
#     except:
#         print('Warning', row, 'baseline_duration')
#     try:
#         persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
#     except:
#         print('Warning', row, 'diff_length')
#     try:
#         persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
#     except:
#         print('Warning', row, 'diff_duration')
#     try:
#         persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'x_activity_from_TR')
#     try:
#         persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'y_activity_From_TR')
#     try:
#         persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'x_activity_To_TR')
#     try:
#         persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'y_activity_To_TR')   
#     try:
#         persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'x_activity_from_Baseline')
#     try:
#         persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'y_activity_From_Baseline')
#     try:
#         persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'x_activity_To_Baseline')
#     try:
#         persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'y_activity_To_Baseline')   
#     try:
#         persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicle Types Used baseline')
#     try:
#         persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicle Types Used TR')
#     try:
#         persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#     except:
#         print('Warning', row, 'Baseline planned mode')
#     try:
#         persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
#     except:
#         print('Warning', row, 'TR planned mode')
#     try:
#         plan_dep_time_baseline = list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']>=plan_dep_time_baseline-1)])[0]
#     except:
#         print('Warning', row, 'mode_choice_baseline')
#     try:
#         persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']>=persons_baseline.at[i,'Planned Depart Time TR']-1)])[0]
#     except:
#         print('Warning', row, 'mode_choice_TR')
        
# ####################################
# for row in PtoPTss_TR_CA_dict.keys():
#     i+=1
#     if i%50==0:
#         print(i)
#         break
        
#     try:
#         persons_baseline.at[i,'Person'] = row[0]
#     except:
#         print('Warning', row, 'person')
#     try:
#         persons_baseline.at[i,'Plan Index'] = int(row[1])
#     except:
#         print('Warning', row, 'plan_index')
#     try:
#         persons_baseline.at[i,'Activity From Baseline'] = int(list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'Activity From Baseline')
#     try:
#         persons_baseline.at[i,'Activity To Baseline'] = int(list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0])
#     except:
#         print('Warning', row, 'Activity To Baseline')
#     try:
#         persons_baseline.at[i,'Activity From TR'] = int(list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'Activity From TR')
#     try:
#         persons_baseline.at[i,'Activity To TR'] = int(list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0])
#     except:
#         print('Warning', row, 'Activity To TR')  
#     try:
#         persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'plan_dep_time_baseline')
#     try:
#         persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'plan_dep_time_TR')
#     try:
#         persons_baseline.at[i,'Project Tried'] = 'CA - Electrification Project'
#     except:
#         print('Warning', row, 'project tried')
#     try:
#         persons_baseline.at[i,'Log Sum Baseline'] = np.unique(list(person_trips_baseline['mode_choice_logsum_y'][(person_trips_baseline['person_id']==row[0])&(person_trips_baseline['trip_num']==row[1])]))
#     except:
#         print('Warning', row, 'Log Sum Baseline')
#     try:
#         persons_baseline.at[i,'Log Sum TR'] = np.unique(list(person_TR['mode_choice_logsum_y'][(person_TR['person_id']==row[0])&(person_TR['trip_num']==row[1])]))
#     except:
#         print('Warning', row, 'Log Sum TR')
#     try:
#         persons_baseline.at[i,'Diff Log Sum'] = persons_baseline.at[i,'Log Sum TR']-persons_baseline.at[i,'Log Sum Baseline']
#     except:
#         print('Warning', row, 'Diff Log Sum TR')
#     try:
#         persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicles Used Baseline')
#     try:
#         persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicles Used TR')
#     try:
#         persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Bus agencies Used Baseline')
#     try:
#         persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Bus agencies Used TR')
#     try:
#         persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
#     except:
#         print('Warning', row, 'First Bus agency Used Baseline')
#     try:
#         persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
#     except:
#         print('Warning', row, 'First Bus agency Used TR')  
#     try:
#         persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
#     except:
#         print('Warning', row, 'TR_length')
#     try:
#         persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
#     except:
#         print('Warning', row, 'TR_duration')
#     try:
#         persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
#     except:
#         print('Warning', row, 'baseline_length')
#     try:
#         persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
#     except:
#         print('Warning', row, 'baseline_duration')
#     try:
#         persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
#     except:
#         print('Warning', row, 'diff_length')
#     try:
#         persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
#     except:
#         print('Warning', row, 'diff_duration')
#     try:
#         persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'x_activity_from_TR')
#     try:
#         persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'y_activity_From_TR')
#     try:
#         persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'x_activity_To_TR')
#     try:
#         persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'y_activity_To_TR')   
#     try:
#         persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'x_activity_from_Baseline')
#     try:
#         persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'y_activity_From_Baseline')
#     try:
#         persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'x_activity_To_Baseline')
#     try:
#         persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'y_activity_To_Baseline')   
#     try:
#         persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicle Types Used baseline')
#     try:
#         persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicle Types Used TR')
#     try:
#         persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#     except:
#         print('Warning', row, 'Baseline planned mode')
#     try:
#         persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
#     except:
#         print('Warning', row, 'TR planned mode')
#     try:
#         plan_dep_time_baseline = list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']>=plan_dep_time_baseline-1)])[0]
#     except:
#         print('Warning', row, 'mode_choice_baseline')
#     try:
#         persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']>=persons_baseline.at[i,'Planned Depart Time TR']-1)])[0]
#     except:
#         print('Warning', row, 'mode_choice_TR')
        
# ####################################
# for row in PtoPTss_TR_AC_dict.keys():
#     i+=1
#     if i%50==0:
#         print(i) 
#         break
        
#     try:
#         persons_baseline.at[i,'Person'] = row[0]
#     except:
#         print('Warning', row, 'person')
#     try:
#         persons_baseline.at[i,'Plan Index'] = int(row[1])
#     except:
#         print('Warning', row, 'plan_index')    
#     try:
#         persons_baseline.at[i,'Activity From Baseline'] = int(list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'Activity From Baseline')
#     try:
#         persons_baseline.at[i,'Activity To Baseline'] = int(list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0])
#     except:
#         print('Warning', row, 'Activity To Baseline')
#     try:
#         persons_baseline.at[i,'Activity From TR'] = int(list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'Activity From TR')
#     try:
#         persons_baseline.at[i,'Activity To TR'] = int(list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0])
#     except:
#         print('Warning', row, 'Activity To TR')   
#     try:
#         persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'plan_dep_time_baseline')
#     try:
#         persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
#     except:
#         print('Warning', row, 'plan_dep_time_TR')
#     try:
#         persons_baseline.at[i,'Project Tried'] = 'AC - 1TEMPO'
#     except:
#         print('Warning', row, 'project tried')
#     try:
#         persons_baseline.at[i,'Log Sum Baseline'] = np.unique(list(person_trips_baseline['mode_choice_logsum_y'][(person_trips_baseline['person_id']==row[0])&(person_trips_baseline['trip_num']==row[1])]))
#     except:
#         print('Warning', row, 'Log Sum Baseline')
#     try:
#         persons_baseline.at[i,'Log Sum TR'] = np.unique(list(person_TR['mode_choice_logsum_y'][(person_TR['person_id']==row[0])&(person_TR['trip_num']==row[1])]))
#     except:
#         print('Warning', row, 'Log Sum TR')
#     try:
#         persons_baseline.at[i,'Diff Log Sum'] = persons_baseline.at[i,'Log Sum TR']-persons_baseline.at[i,'Log Sum Baseline']
#     except:
#         print('Warning', row, 'Diff Log Sum TR')
#     try:
#         persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicles Used Baseline')
#     try:
#         persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicles Used TR')    
#     try:
#         persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Bus agencies Used Baseline')
#     try:
#         persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Bus agencies Used TR')
#     try:
#         persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
#     except:
#         print('Warning', row, 'First Bus agency Used Baseline')
#     try:
#         persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
#     except:
#         print('Warning', row, 'First Bus agency Used TR')
#     try:
#         persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
#     except:
#         print('Warning', row, 'TR_length')
#     try:
#         persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
#     except:
#         print('Warning', row, 'TR_duration')
#     try:
#         persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
#     except:
#         print('Warning', row, 'baseline_length')
#     try:
#         persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
#     except:
#         print('Warning', row, 'baseline_duration')
#     try:
#         persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
#     except:
#         print('Warning', row, 'diff_length')
#     try:
#         persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
#     except:
#         print('Warning', row, 'diff_duration')
#     try:
#         persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'x_activity_from_TR')
#     try:
#         persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'y_activity_From_TR')
#     try:
#         persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'x_activity_To_TR')
#     try:
#         persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'y_activity_To_TR')   
#     try:
#         persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'x_activity_from_Baseline')
#     try:
#         persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#     except:
#         print('Warning', row, 'y_activity_From_Baseline')
#     try:
#         persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'x_activity_To_Baseline')
#     try:
#         persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
#     except:
#         print('Warning', row, 'y_activity_To_Baseline')   
#     try:
#         persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicle Types Used baseline')
#     try:
#         persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
#     except:
#         print('Warning', row, 'Vehicle Types Used TR')
#     try:
#         persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#     except:
#         print('Warning', row, 'Baseline planned mode')
#     try:
#         persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
#     except:
#         print('Warning', row, 'TR planned mode')
#     try:
#         plan_dep_time_baseline = list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']>=plan_dep_time_baseline-1)])[0]
#     except:
#         print('Warning', row, 'mode_choice_baseline')
#     try:
#         persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']>=persons_baseline.at[i,'Planned Depart Time TR']-1)])[0]
#     except:
#         print('Warning', row, 'mode_choice_TR')

        
# zipcode = gpd.read_file('/Users/cpoliziani/Downloads/TAZ to ZIP/ZCTA2010.shp')
# persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity From TR', 'Y Activity From TR', 'ZIP Departure TR')
# persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity To TR', 'Y Activity To TR', 'ZIP Arrival TR')
# persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity From Baseline', 'Y Activity From Baseline', 'ZIP Departure Baseline')
# persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity To Baseline', 'Y Activity To Baseline', 'ZIP Arrival Baseline')


# #Add Switch From Column
# switch_type = []
# for agencies_baseline, project in zip(persons_baseline['Bus agencies Used Baseline'],persons_baseline['Project Tried']):
#     if len(agencies_baseline) == 0:
#         switch_type.append('Switch from another mode')
#     elif project == 'SF - Central Subway' and 'SF' in agencies_baseline:
#         switch_type.append('Switch from same transit agency')
#     elif project == 'AC - 1TEMPO' and 'AC' in agencies_baseline:
#         switch_type.append('Switch from same transit agency')
#     elif project == 'CA - Electrification Project' and 'ca' in agencies_baseline:
#         switch_type.append('Switch from same transit agency')
#     else:
#         switch_type.append('Switch from another transit agency')

# persons_baseline['Switch From'] = switch_type

# persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')


# person_trips_baseline





# persons_baseline_AC = persons_baseline[persons_baseline['Project Tried']=='AC - 1TEMPO']

# persons_baseline_AC.dropna(axis=0)

# sum(persons_baseline_AC['Diff Duration'])

# sum(persons_baseline_AC['Diff Length'])







# events = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz', nrows = 900000)




# veh = events[['vehicle','type']].dropna()
# veh = veh[(veh['vehicle'].str.contains('rideHail'))&(veh['type']=='PathTraversal')]
# vei = []
# for ve in veh['vehicle']:
#     vei.append(ve[-10:])

# np.unique(vei, return_counts=True)

# pd.read_csv('/Users/cpoliziani/Downloads/0.skimsRidehail (1).csv.gz')












# # for row in PtoPTss_TR_CA_dict.keys():
# #     i+=1
# #     if i%1000==0:
# #         print(i)
# #     try:
# #         persons_baseline.at[i,'person'] = row[0]
# #         persons_baseline.at[i,'plan_index'] = row[1]
# #         persons_baseline.at[i,'project tried'] = 'CA - Electrification Project'
# #         persons_baseline.at[i,'TR_length'] = PtoPTss_TR_dict[row][0]
# #         persons_baseline.at[i,'TR_duration'] = PtoPTss_TR_dict[row][1]
# #         persons_baseline.at[i,'baseline_length'] = PtoPTss_baseline_dict[row][0]
# #         persons_baseline.at[i,'baseline_duration'] = PtoPTss_baseline_dict[row][1]
# #         persons_baseline.at[i,'diff_length'] = persons_baseline.at[i,'TR_length']-persons_baseline.at[i,'baseline_length']
# #         persons_baseline.at[i,'diff_duration'] = persons_baseline.at[i,'TR_duration']-persons_baseline.at[i,'baseline_duration']
# #         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
# #     except:
# #         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
# #         persons_baseline.at[i,'person'] = row[0]
# #         persons_baseline.at[i,'plan_index'] = row[1]
# #         persons_baseline.at[i,'project tried'] = 'CA - Electrification Project'
# #         print('WARNING:',row[0], row[1]) 


# # for row in PtoPTss_TR_AC_dict.keys():
# #     i+=1
# #     if i%1000==0:
# #         print(i)
# #     try:
# #         persons_baseline.at[i,'person'] = row[0]
# #         persons_baseline.at[i,'plan_index'] = row[1]
# #         persons_baseline.at[i,'project tried'] = 'AC - 1 Tempo BRT Line'
# #         persons_baseline.at[i,'TR_length'] = PtoPTss_TR_dict[row][0]
# #         persons_baseline.at[i,'TR_duration'] = PtoPTss_TR_dict[row][1]
# #         persons_baseline.at[i,'baseline_length'] = PtoPTss_baseline_dict[row][0]
# #         persons_baseline.at[i,'baseline_duration'] = PtoPTss_baseline_dict[row][1]
# #         persons_baseline.at[i,'diff_length'] = persons_baseline.at[i,'TR_length']-persons_baseline.at[i,'baseline_length']
# #         persons_baseline.at[i,'diff_duration'] = persons_baseline.at[i,'TR_duration']-persons_baseline.at[i,'baseline_duration']
# #         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
# #     except:
# #         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
# #         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
# #         persons_baseline.at[i,'person'] = row[0]
# #         persons_baseline.at[i,'plan_index'] = row[1]
# #         persons_baseline.at[i,'project tried'] = 'AC - 1 Tempo BRT Line'
# #         print('WARNING:',row[0], row[1])




# ##############
# # trips_TR = pd.concat([line_CA_trips_TR['trip_id'],line_CS_trips_TR['trip_id'],line_1T_trips_TR['trip_id']])
# # for vehicle, person, plan_index, vehicle2 in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'],PtoPTss['vehicle2'] ):
# #     if int(vehicle.split(':')[1]) in list(trips_TR):
# #         i+=1
# #         if i%1000==0:
# #             print(i)
# #         try:

# #             persons_baseline.at[i,'person'] = person
# #             persons_baseline.at[i,'plan_index'] = plan_index
# #             persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
# #             persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
# #             persons_baseline.at[i,'vehicle_now'] = vehicle
# #             if vehicle2 == 'SF':
# #                 persons_baseline.at[i,'project'] = 'CS'
# #             if vehicle2 == 'Ca':
# #                 persons_baseline.at[i,'project'] = 'CA'
# #             if vehicle2 == 'AC':
# #                 persons_baseline.at[i,'project'] = '1T'

# #             persons_baseline.at[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
# #             persons_baseline.at[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
# #             persons_baseline.at[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
# #             persons_baseline.at[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
# #             persons_baseline.at[i,'diff_speed'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
# #         except:
# #             persons_baseline.at[i,'person'] = person
# #             persons_baseline.at[i,'plan_index'] = plan_index
# #             persons_baseline.at[i,'vehicle_now'] = vehicle
# #             if vehicle2 == 'SF':
# #                 persons_baseline.at[i,'project'] = 'CS'
# #             if vehicle2 == 'Ca':
# #                 persons_baseline.at[i,'project'] = 'CA'
# #             if vehicle2 == 'AC':
# #                 persons_baseline.at[i,'project'] = '1T'

# #             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)
# #     print('Warning')




# # persons_baseline = pd.DataFrame(['person','plan_index','mode_baseline','vehicle_baseline','agency_baseline',
# #                                  'x_activity_TR','y_activity_TR','vehicle_now'])
# # i = 0
# # PtoPTss=pd.concat(PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR)
# # for vehicle, person, plan_index in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'] ):
# #     if int(vehicle.split(':')[1]) in list(line_1T_trips_TR):
# #         i+=1
# #         persons_baseline.loc[i,'person'] = person
# #         persons_baseline.loc[i,'plan_index'] = plan_index
# #         persons_baseline.loc[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
# #         persons_baseline.loc[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
# #         persons_baseline.loc[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
# #         persons_baseline.loc[i,'vehicle_now'] = '1T'
# #         try:
# #             persons_baseline.loc[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.loc[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.loc[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.loc[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.loc[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.loc[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.loc[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
# #             persons_baseline.loc[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
# #             persons_baseline.loc[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
# #             persons_baseline.loc[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
# #             persons_baseline.loc[i,'speed_length'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
# #         except:
# #             persons_baseline.loc[i,'vehicle_baseline'] = None
# #             persons_baseline.loc[i,'agency_baseline'] = None
# #             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)


# # persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')

# # PtoPTss_baseline


# # persons_baseline = pd.DataFrame()
# # i = 0
# # PtoPTss=pd.concat([PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR])
# # for vehicle, person, plan_index, vehicle2 in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'],PtoPTss['vehicle2'] ):
# #     if int(vehicle.split(':')[1]) in list(line_1T_trips_TR):
# #         i+=1

# #             persons_baseline.at[i,'person'] = person
# #             persons_baseline.at[i,'plan_index'] = plan_index
# #             persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
# #             persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
# #             persons_baseline.at[i,'vehicle_now'] = vehicle
# #             if vehicle2 == 'SF':
# #                 persons_baseline.at[i,'project'] = 'CS'
# #             if vehicle2 == 'Ca':
# #                 persons_baseline.at[i,'project'] = 'CA'
# #             if vehicle2 == 'AC':
# #                 persons_baseline.at[i,'project'] = '1T'
# #         try:

# #             persons_baseline.at[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
# #             persons_baseline.at[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
# #             persons_baseline.at[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
# #             persons_baseline.at[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
# #             persons_baseline.at[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
# #             persons_baseline.at[i,'diff_speed'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
# #         except:
# #             persons_baseline.at[i,'person'] = person
# #             persons_baseline.at[i,'plan_index'] = plan_index
# #             persons_baseline.at[i,'vehicle_now'] = vehicle
# #             if vehicle2 == 'SF':
# #                 persons_baseline.at[i,'project'] = 'CS'
# #             if vehicle2 == 'Ca':
# #                 persons_baseline.at[i,'project'] = 'CA'
# #             if vehicle2 == 'AC':
# #                 persons_baseline.at[i,'project'] = '1T'

# #             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)


# # persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')








# LIRR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2020LIRR.csv')
# LIRR_capacities_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2020LIRR.csv')
# LIRR_capacities_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2021LIRR.csv')
# LIRR_capacities_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/jan2022LIRR.csv')
# LIRR_capacities_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2022LIRR.csv')

# MNR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2020MNR.csv')
# MNR_capacities_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2020MNR.csv')
# MNR_capacities_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2021MNR.csv')
# MNR_capacities_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/jan2022MNR.csv')
# MNR_capacities_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2022MNR.csv')

# # LIRR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/LIRR_capacities.csv')
# # MNR_capacities = MNR_capacities.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
# # LIRR_capacities = LIRR_capacities.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()


# capacities_data = [LIRR_capacities_apr2020,
#         LIRR_capacities_aug2020,
#         LIRR_capacities_aug2021,
#         LIRR_capacities_jan2022,
#         LIRR_capacities_may2022,
#         MNR_capacities_apr2020,
#         MNR_capacities_aug2020,
#         MNR_capacities_aug2021,
#         MNR_capacities_jan2022,
#         MNR_capacities_may2022,
#         ]

# for capacity_data in capacities_data:
#     trains = []
#     for train in capacity_data['Train']:
#         try:
#             trains.append(int(train))
#         except:
#             trains.append(train)
#     capacity_data['Train'] = trains
    


# LIRR_capacities_apr2020 = LIRR_capacities_apr2020.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
# LIRR_capacities_aug2020 = LIRR_capacities_aug2020.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
# LIRR_capacities_aug2021 = LIRR_capacities_aug2021.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
# LIRR_capacities_jan2022 = LIRR_capacities_jan2022.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
# LIRR_capacities_may2022 = LIRR_capacities_may2022.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()

# MNR_capacities_apr2020 = MNR_capacities_apr2020.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
# MNR_capacities_aug2020 = MNR_capacities_aug2020.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
# MNR_capacities_aug2021 = MNR_capacities_aug2021.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
# MNR_capacities_jan2022 = MNR_capacities_jan2022.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
# MNR_capacities_may2022 = MNR_capacities_may2022.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()


# LIRR_trips_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Long_Island_Rail_20200318/trips.txt')
# LIRR_trips_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Long_Island_Rail_20200629/trips.txt')
# LIRR_trips_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Long_Island_Rail_20210726/trips.txt')
# LIRR_trips_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Long_Island_Rail_20211216/trips.txt')
# LIRR_trips_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Long_Island_Rail_20220430/trips.txt')

# LIRR_trips_apr2020['agency_id']='LI'
# LIRR_trips_aug2020['agency_id']='LI'
# LIRR_trips_aug2021['agency_id']='LI'
# LIRR_trips_jan2022['agency_id']='LI'
# LIRR_trips_may2022['agency_id']='LI'

# MNR_trips_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Metro-North_Railroad_20200325/trips.txt')
# MNR_trips_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Metro-North_Railroad_20200731/trips.txt')
# MNR_trips_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Metro-North_Railroad_20210721/trips.txt')
# MNR_trips_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Metro-North_Railroad_20211222/trips.txt')
# MNR_trips_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Metro-North_Railroad_20220429/trips.txt')

# MNR_trips_apr2020['agency_id']='1'
# MNR_trips_aug2020['agency_id']='1'
# MNR_trips_aug2021['agency_id']='1'
# MNR_trips_jan2022['agency_id']='1'
# MNR_trips_may2022['agency_id']='1'

# # MNR_routes_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Metro-North_Railroad_20200325/routes.txt')
# # MNR_routes_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Metro-North_Railroad_20200731/routes.txt')
# # MNR_routes_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Metro-North_Railroad_20210721/routes.txt')
# # MNR_routes_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Metro-North_Railroad_20211222/routes.txt')
# # MNR_routes_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Metro-North_Railroad_20220429/routes.txt')

# # trips_data_MNR = [MNR_trips_apr2020,
# #         MNR_trips_aug2020,
# #         MNR_trips_aug2021,
# #         MNR_trips_jan2022,
# #         MNR_trips_may2022,
# #         ]

# # routes_data_MNR = [MNR_routes_apr2020,
# #         MNR_routes_aug2020,
# #         MNR_routes_aug2021,
# #         MNR_routes_jan2022,
# #         MNR_routes_may2022,
# #         ]

# # for trip_data_MNR, route_data_MNR, i  in zip(trips_data_MNR,routes_data_MNR, range(len(trips_data))):
# #     agencies =[]
# #     for route in trip_data_MNR['route_id']:
# #         agencies.append(list(route_data_MNR['agency_id'][route_data_MNR['route_id']==route])[0])
# #     trip_data_MNR['agency_id']=agencies


# # MNR_trips_apr2020 = trips_data_MNR[0]
# # MNR_trips_aug2020 = trips_data_MNR[1]
# # MNR_trips_aug2021 = trips_data_MNR[2]
# # MNR_trips_jan2022 = trips_data_MNR[3]
# # MNR_trips_may2022 = trips_data_MNR[4]

# trips_data = [LIRR_trips_apr2020,
#         LIRR_trips_aug2020,
#         LIRR_trips_aug2021,
#         LIRR_trips_jan2022,
#         LIRR_trips_may2022,
#         MNR_trips_apr2020,
#         MNR_trips_aug2020,
#         MNR_trips_aug2021,
#         MNR_trips_jan2022,
#         MNR_trips_may2022,
#         ]

# capacities_data = [LIRR_capacities_apr2020,
#         LIRR_capacities_aug2020,
#         LIRR_capacities_aug2021,
#         LIRR_capacities_jan2022,
#         LIRR_capacities_may2022,
#         MNR_capacities_apr2020,
#         MNR_capacities_aug2020,
#         MNR_capacities_aug2021,
#         MNR_capacities_jan2022,
#         MNR_capacities_may2022,
#         ]

# for trip_data, capacity_data in zip(trips_data, capacities_data):

#     capacities = []
#     wrong_ids = []
#     print('number of trip', len(trip_data),'number of capacities', len(capacity_data))
#     print('number of unique trip', len(np.unique(trip_data['trip_short_name'])))
#     for trip_short_name in trip_data['trip_short_name']:
#         try:
#             capacities.append(capacity_data[trip_short_name][0][0])
#         except:
#             try:
#                 capacities.append(capacity_data[float(trip_short_name)][0][0])
#             except:
#                 wrong_ids.append(trip_short_name)
#                 capacities.append(np.nan)
#     trip_data['capacity'] = capacities
#     print('Warning!! Not found',len(wrong_ids))
# #     print(trip_data)
#     print('######################')



# for trip_data in trips_data:
#     max_cap = max(list(trip_data['capacity'].dropna()))
#     print(max_cap)
#     trip_data['capacity'] = trip_data['capacity'].fillna(max_cap)




# trips_data_apr2020 = [trips_data[0],trips_data[5]]
# trips_data_aug2020 = [trips_data[1],trips_data[6]]
# trips_data_aug2021 = [trips_data[2],trips_data[7]]
# trips_data_jan2022 = [trips_data[3],trips_data[8]]
# trips_data_may2022 = [trips_data[4],trips_data[9]]

# transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
#                                            'routeId',
#                                            'tripId',
#                                            'capacity',
#                                             'vehicleTypeId',
#                                           ])
# i=0      
# for trip_data, j in zip(trips_data_apr2020,[0,1]):
#     for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
#         i+=1
#         if i%5000==0:
#             print(i)
#         transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
#         transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
#         if j == 0:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20200318:'+str(tripID)
#         elif j == 1:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20200325:'+str(tripID)
#         transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
#         transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

# transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_apr2020.csv')


# transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
#                                            'routeId',
#                                            'tripId',
#                                            'capacity',
#                                             'vehicleTypeId',
#                                           ])
# i=0      
# for trip_data, j in zip(trips_data_aug2020,[0,1]):
#     for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
#         i+=1
#         if i%5000==0:
#             print(i)
#         transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
#         transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
#         if j == 0:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20200629:'+str(tripID)
#         elif j == 1:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20200731:'+str(tripID)
#         transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
#         transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

# transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_aug2020.csv')



# transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
#                                            'routeId',
#                                            'tripId',
#                                            'capacity',
#                                             'vehicleTypeId',
#                                           ])
# i=0      
# for trip_data, j in zip(trips_data_aug2021,[0,1]):
#     for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
#         i+=1
#         if i%5000==0:
#             print(i)
#         transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
#         transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
#         if j == 0:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20210726:'+str(tripID)
#         elif j == 1:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20210721:'+str(tripID)
#         transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
#         transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

# transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_aug2021.csv')



# transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
#                                            'routeId',
#                                            'tripId',
#                                            'capacity',
#                                             'vehicleTypeId',
#                                           ])
# i=0      
# for trip_data, j in zip(trips_data_jan2022,[0,1]):
#     for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
#         i+=1
#         if i%5000==0:
#             print(i)
#         transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
#         transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
#         if j == 0:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20211216:'+str(tripID)
#         elif j == 1:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20211222:'+str(tripID)
#         transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
#         transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

# transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_jan2022.csv')



# transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
#                                            'routeId',
#                                            'tripId',
#                                             'capacity',
#                                            'vehicleTypeId',
#                                           ])
# i=0      
# for trip_data, j in zip(trips_data_may2022,[0,1]):
#     for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
#         i+=1
#         if i%5000==0:
#             print(i)
#         transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
#         transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
#         if j == 0:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20220430:'+str(tripID)
#         elif j == 1:
#             transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20220429:'+str(tripID)
#         transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
#         transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

# transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_may2022.csv')



# trips_data_apr2020[1]

# pd.read_csv('s3://beam-outputs/output/newyork/new-york-jan2022-0-of-10__2022-09-21_17-07-49_qdx/ITERS/it.10/10.events.csv.gz', nrows = 90000)




